In [1]:
import pandas as pd
import numpy as np
import random
from os import listdir
from os.path import isfile, join,getsize,isdir
import csv
import numpy
import cPickle as pickle
import json
from random import shuffle
import warnings
import time
#import msgpack
import re
import matplotlib.pyplot as plt
from datetime import date
import math
import pylab
import pandas_profiling
import unicodedata
import dateutil.relativedelta

#n'affiche pas les warnings
warnings.filterwarnings('ignore')
#affiche la totalité des colonnes du dataframe
pd.set_option('display.max_columns', None)
#path = "C:\Users\Data Science 5\Desktop\L_equipe"
path = "W:\L_equipe"


### chargement des dataframes

In [4]:
#df_client = pd.read_pickle(r"W:\L_equipe\s_dataframe_general_xiti_clients.pkl")
#df_prod42 = pd.read_pickle(path+"\s_dataframe_general_vel_PROD42.pkl")
#df_client_clean = pd.read_pickle(path+"\s_dataframe_client_clean_final.pkl")
df_client = pd.read_pickle(r"W:\L_equipe\df_client_true_site.pkl")
#dataframe = pd.read_pickle(r"C:\Users\Data Science 5\Desktop\L_equipe\df_client_true_site.pkl")

#df_compte = pd.read_pickle(r"W:\L_equipe\s_dataframe_general_vel_compte.pkl")
#df_commande = pd.read_pickle(r"W:\L_equipe\s_dataframe_general_vel_commande.pkl")
#df_prod42_clean = pd.read_pickle(path+"\s_dataframe_prod42_clean.pkl")
#df_compte_clean = pd.read_pickle(r"W:\L_equipe\s_dataframe_compte_clean.pkl")
#df_commande_clean = pd.read_pickle(r"W:\L_equipe\s_dataframe_commande_clean.pkl")
#df_pages_juin_p1 = pd.read_pickle(r"W:\L_equipe\s_dataframe_general_xiti_pages_juin2015_partie1.pkl")
#df_pages_juin_p1 = pd.read_pickle("s_dataframe_pages_juin_partie1_clean.pkl")
#df_session_juin = pd.read_pickle(path+"\s_dataframe_general_xiti_sessions_juin2015.pkl")
#df_session_juin_clean = pd.read_pickle(r"W:\L_equipe\s_dataframe_session_juin2015_clean.pkl")

### Clean sites

In [14]:
#on ne garde que les sites de l'équipe et les client qui sont dans la base Client
#----------------------------------Sites de l'equipe ----------------------------------
df_client = pd.read_pickle(r"W:\L_equipe\df_client_true_site.pkl")

def nettoyage_site(dataframe) :
    dataframe = dataframe.loc[dataframe['id_site'].isin([492987,496306,496307,496838,502193,509043,539121,548647])]
    liste_client = dataframe['id_client'].tolist()
    return dataframe, liste_client 

df_client, liste_client = nettoyage_site(df_client)
#save
#df_client.to_pickle(r"C:\Users\Data Science 5\Desktop\L_equipe\df_client_true_site.pkl")

### Clean subs

In [8]:
#--------------------nettoyage nouveaux abonnés--------------------------
path = "W:\L_Equipe"
df_prod42 = pd.read_pickle(path+"\dataframes\s_dataframe_general_vel_PROD42.pkl")

def nettoyage_sub(dataframe, liste_client) :
    #selection des abonnés de l'equipe
    dataframe = dataframe.loc[dataframe['ClientUserId'].isin(liste_client)]
    #formatage datetime 
    dataframe['SubscriptionCreated'] = pd.to_datetime(dataframe['SubscriptionCreated'], format='%d/%m/%Y %H:%M:%S')
    dataframe['SubscriptionLastUpdated'] = pd.to_datetime(dataframe['SubscriptionLastUpdated'], format='%d/%m/%Y %H:%M:%S')
    dataframe['ServiceExpiry'] = pd.to_datetime(dataframe['ServiceExpiry'], format='%d/%m/%Y %H:%M:%S')
    
    #on ne garde que les abonnés de l'année 2015
    dataframe['anciennete_souscripteur'] = dataframe.groupby('ClientUserId').SubscriptionCreated.transform('min')
    dataframe = dataframe.loc[(dataframe.anciennete_souscripteur > date(2015,1,1)) & (dataframe.anciennete_souscripteur < date(2015,7,1)) ]
    #de plus on ne prends pas en compte les mises a jour de juillet 2015
    dataframe = dataframe.loc[(dataframe.SubscriptionLastUpdated > date(2015,1,1)) & (dataframe.SubscriptionLastUpdated < date(2015,7,1))]
    
    #correction fautes de frapes et autres modifs
    dataframe.loc[(dataframe.ServiceTitle == 'SFR_Full_Extra_Suplementaire'), 'ServiceTitle']= 'SFR_Full_Extra_Supplementaire'
    dataframe.loc[(dataframe.ServiceTitle == 'Fete_des_peres_2015_v2'), 'ServiceTitle']= 'Fete_des_peres_2015'
    dataframe.loc[(dataframe.ServiceTitle == 'Running_heroes_1M_gratuit'), 'ServiceTitle']= 'Running_Heroes_1M_gratuit'
    dataframe.loc[(dataframe.ServiceTitle == 'TEST_ROLAND_GARROS_'), 'ServiceTitle']= 'RG2015_1€_15jrs_puis_11.99€'
    dataframe.loc[(dataframe.ServiceTitle == 'TEMPLATE_ROLAND_GARROS'), 'ServiceTitle']= 'RG2015_1€_15jrs_puis_11.99€'
    dataframe.loc[(dataframe.ServiceTitle == 'CocaCola_Total_1mois_essai '), 'ServiceTitle']= 'CocaCola_Total_1mois_essai'
    
    #fusion du nettoyage des souscriptions avec les abonnés
    df_sous = pd.read_excel(path+'\souscription_detail.xlsx')
    df_sous.drop(['ServiceTitle'], axis=1, inplace=True)
    dataframe = pd.merge(dataframe, df_sous, how='inner', on='ServiceID')
    
    #group by
    #dataframe['somme_prix'] = dataframe.groupby(['ClientUserId', 'SubscriptionId'])['ExplicitPrice'].transform('sum')
    dataframe['SubscriptionCreated'] = dataframe.groupby(['ClientUserId','SubscriptionId'])["SubscriptionCreated"].transform('min')
    dataframe['SubscriptionLastUpdated'] = dataframe.groupby(['ClientUserId', 'SubscriptionId'])["SubscriptionLastUpdated"].transform('max')
    dataframe['ServiceExpiry'] = dataframe.groupby(['ClientUserId', 'SubscriptionId'])['ServiceExpiry'].transform('max')
    
    #feature generation
    dataframe = dataframe.loc[((dataframe.ServiceGroupTitle == 'Abo EQP') | (dataframe.ServiceGroupTitle == "Abo EQP via Tiers"))]
    dataframe.loc[((dataframe['ServiceGroupTitle'] == 'Abo EQP') & (dataframe.prix_mensuel != 0)), 'service'] = 'Abo_Equipe_payant' 
    dataframe.loc[((dataframe['ServiceGroupTitle'] == 'Abo EQP') & (dataframe.prix_mensuel == 0)), 'service'] = 'Abo_Equipe_gratuit'
    dataframe.loc[((dataframe['ServiceGroupTitle'] == 'Abo EQP via Tiers') & (dataframe.prix_mensuel != 0)), 'service'] = 'Abo_via_tiers_payant' 
    dataframe.loc[((dataframe['ServiceGroupTitle'] == 'Abo EQP via Tiers') & (dataframe.prix_mensuel == 0)), 'service'] = 'Abo_via_tiers_gratuit' 
          
    dataframe['provider'] = dataframe['UKI'].str.extract(('.*@(.+)\..*'))
    
    
    dataframe["sub_month"] = numpy.nan
    dataframe.loc[(dataframe.anciennete_souscripteur > date(2015,1,1)) 
                  & (dataframe.anciennete_souscripteur < date(2015,2,1)), 'sub_month'] = '1'
    dataframe.loc[(dataframe.anciennete_souscripteur > date(2015,2,1)) 
                  & (dataframe.anciennete_souscripteur < date(2015,3,1)), 'sub_month'] = '2'
    dataframe.loc[(dataframe.anciennete_souscripteur > date(2015,3,1)) 
                  & (dataframe.anciennete_souscripteur < date(2015,4,1)), 'sub_month'] = '3'
    dataframe.loc[(dataframe.anciennete_souscripteur > date(2015,4,1)) 
                  & (dataframe.anciennete_souscripteur < date(2015,5,1)), 'sub_month'] = '4'
    dataframe.loc[(dataframe.anciennete_souscripteur > date(2015,5,1)) 
                  & (dataframe.anciennete_souscripteur < date(2015,6,1)), 'sub_month'] = '5'
    dataframe.loc[(dataframe.anciennete_souscripteur > date(2015,6,1)) 
                  & (dataframe.anciennete_souscripteur < date(2015,7,1)), 'sub_month'] = '6'
        
    dataframe["anciennete_souscripteur"] = (date(2015, 7, 1) - dataframe["anciennete_souscripteur"])
    dataframe['duree_abonnement_reelle'] = dataframe['ServiceExpiry'] - dataframe['SubscriptionCreated']
    dataframe["derniere_maj"] = (date(2015, 7, 1) - dataframe["SubscriptionLastUpdated"])
    dataframe.anciennete_souscripteur= pd.to_numeric(dataframe.anciennete_souscripteur)/(1000000000 * 60 * 60 * 24)
    dataframe.derniere_maj = pd.to_numeric(dataframe.derniere_maj)/(1000000000 * 60 * 60 * 24)
    dataframe.duree_abonnement_reelle = pd.to_numeric(dataframe.duree_abonnement_reelle)/(1000000000 * 60 * 60 * 24)
    
    dataframe['somme_mensualites'] = ((np.maximum(0,dataframe['duree_abonnement_reelle']
                                                  -dataframe["duree_cadeau"])/30.416)*dataframe["prix_mensuel"]
                                     + (dataframe["duree_cadeau"]/30.416)*dataframe["prix_mensuel_cadeau"])
    
    #sauvegarde d'un temp pour la data vis
    dataframe.to_pickle(path+"\dataframes\s_dataframe_temp_sub_2015.pkl")
    
    #dummies 
    dataframe = pd.get_dummies(dataframe, columns = ['SubscriptionStatus'], prefix ='SubStatus' )
    dataframe = pd.get_dummies(dataframe, columns = ['AutoRenew'], prefix ='AutoRenew' )
    dataframe = pd.get_dummies(dataframe, columns = ['service'], prefix = 'service')
    dataframe = pd.get_dummies(dataframe , columns=['sub_month'], prefix = 'sub_month')
    
    #group by pour traiter certaines dummies
    dataframe.rename(columns=lambda x: x.replace(" ", "_"), inplace=True)
    dataframe['SubStatus_Active_Subscription'] = dataframe.groupby(
        ['ClientUserId','SubscriptionId'])["SubStatus_Active_Subscription"].transform('min')
    dataframe['SubStatus_Cancelled_By_AutoRenew_Process'] = dataframe.groupby(
        ['ClientUserId','SubscriptionId'])["SubStatus_Cancelled_By_AutoRenew_Process"].transform('max')
    dataframe['SubStatus_Cancelled_By_Customer_Support_Agent'] = dataframe.groupby(
        ['ClientUserId','SubscriptionId'])["SubStatus_Cancelled_By_Customer_Support_Agent"].transform('max')
    dataframe['SubStatus_Cancelled_By_User'] = dataframe.groupby(
        ['ClientUserId','SubscriptionId'])["SubStatus_Cancelled_By_User"].transform('max')
    dataframe['SubStatus_Expired_Subscription'] = dataframe.groupby(
        ['ClientUserId','SubscriptionId'])["SubStatus_Expired_Subscription"].transform('max')
    dataframe['SubStatus_Failure_Retry_Mode'] = dataframe.groupby(
        ['ClientUserId','SubscriptionId'])["SubStatus_Failure_Retry_Mode"].transform('max')
    dataframe['duree_abonnement_reelle'] = dataframe.groupby(
        ['ClientUserId','SubscriptionId'])["duree_abonnement_reelle"].transform('max')
    dataframe["AutoRenew_0.0"] = dataframe.groupby(
        ['ClientUserId','SubscriptionId'])["AutoRenew_0.0"].transform('max')
    dataframe["AutoRenew_1.0"] = dataframe.groupby(
        ['ClientUserId','SubscriptionId'])["AutoRenew_1.0"].transform('max')
    dataframe["AutoRenew_2.0"] = dataframe.groupby(
        ['ClientUserId','SubscriptionId'])["AutoRenew_2.0"].transform('max')
    dataframe["AutoRenew_4.0"] = dataframe.groupby(
        ['ClientUserId','SubscriptionId'])["AutoRenew_4.0"].transform('max')
    
    #drop
    dataframe.drop(['ExplicitCurrency', 'PriceBandPaymentType', 'PriceBandAmount', 'PriceBandCurrency', 'IsTrial'
                  ,'ExplicitPaymentType',"ServiceGroup", 'ServiceGroupDescription',"UKI"
                      ,"SubscriptionStatusID","ServiceDescription","ServiceExpiry",'ServiceGroupTitle',"ExplicitPrice"
                   ,"SubscriptionLastUpdated", 'ServiceTitle', 'cadeau',], axis=1, inplace=True)#"SubscriptionCreated" ,
    
    dataframe = dataframe.drop_duplicates()
    return dataframe


df_prod42_test = nettoyage_sub(df_prod42, liste_client)
#print df_prod42_test.somme_mensualites.sum()
#df_prod42_test.sort_values(by=['ClientUserId'])
#sauvegarde 
df_prod42_test.to_pickle(path+"\dataframes\s_dataframe_final_sub_2015.pkl")

### Selection de la population des subscribers 

In [ ]:
#----------------------nouveaux abonnés juin------------------------
df_prod42_test = pd.read_pickle(path+"\s_dataframe_final_sub_2015.pkl")
#l_prod42 = df_prod42_test.ClientUserId.unique()

#sub juin 
nouveaux_sub_juin = df_prod42_test.loc[df_prod42_test.sub_month_6 == 1]
nouveaux_sub_juin = nouveaux_sub_juin.loc[nouveaux_sub_juin.somme_mensualites >0]
nouveaux_sub_juin.drop(['sub_month_1', 'sub_month_2', 'sub_month_3'
            , 'sub_month_4','sub_month_5',"sub_month_6"], axis=1, inplace=True)

nouveaux_sub_juin.to_pickle(path+"\s_dataframe_final_sub_payant_juin_2015.pkl")
#
#print nouveaux_sub_juin.shape #-> (2021, 28)
#print len (nouveaux_sub_juin.ClientUserId.unique()) #-> 2011
#
#test1 = nouveaux_sub_juin.loc[((nouveaux_sub_juin.service_Abo_Equipe_payant ==1 ) | (nouveaux_sub_juin.service_Abo_via_tiers_payant == 1))]
#print test1.shape #-> (946, 28)
#print len(test1.ClientUserId.unique()) #-> 945
#
#test2 = nouveaux_sub_juin.loc[nouveaux_sub_juin.somme_mensualites >0]
#print test2.shape #-> (439, 28)
#print len(test2.ClientUserId.unique()) #-> 439
## ***.to_pickle(path+"\s_final_groupby_dataframe_sub_juin_2015.pkl")

### Clean Clients 

In [44]:
#--------------------nettoyage clients--------------------------
#dataframe = pd.read_pickle(r"W:\L_equipe\df_client_true_site.pkl")
dataframe = pd.read_csv(r"V:\L_Equipe\Projet_01042016\tables_brutes\clients_2015.csv")

#dataframe = dataframe.loc[dataframe['id_client'].isin(liste_client)]

#formatage datetime 
dataframe['premiere_visite'] = pd.to_datetime(dataframe['premiere_visite'], format='%Y-%m-%d %H:%M:%S')
dataframe['derniere_visite'] = pd.to_datetime(dataframe['derniere_visite'], format='%Y-%m-%d %H:%M:%S')
#group by
dataframe['somme_visites_totale'] = dataframe.groupby(['id_client']).visites.transform('sum')
dataframe['somme_pages_vues_totale'] = dataframe.groupby(["id_client"])["pages_vues"].transform('sum')
dataframe['anciennete'] = dataframe.groupby(['id_client'])["premiere_visite"].transform('min')
dataframe['recence_de_visite'] = dataframe.groupby(['id_client'])["derniere_visite"].transform('max')
    
#on choisit les clients qui on eu une acitivité entre  mars et mai 2015
#dataframe = dataframe.loc[(dataframe.anciennete < date(2015,6,1)) & (dataframe.recence_de_visite > date(2015,2,28))]
#dataframe = dataframe.loc[dataframe.recence_de_visite > date(2015,2,28)]


dataframe['recence_de_visite'] =(date(2015, 7, 1) - dataframe.recence_de_visite)
dataframe.recence_de_visite = pd.to_numeric(dataframe.recence_de_visite)/(1000000000 * 60 * 60 * 24)
dataframe['anciennete'] =(date(2015, 7, 1) - dataframe.anciennete)
dataframe.anciennete = pd.to_numeric(dataframe.anciennete)/(1000000000 * 60 * 60 * 24)

dataframe['somme_visites_par_site'] = dataframe.groupby(["id_client","id_site"])["visites"].transform('sum')
dataframe['pourcentage_visites_par_site'] = dataframe["somme_visites_par_site"]/dataframe["somme_visites_totale"]*100

dataframe['somme_pages_vues_par_site'] = dataframe.groupby(["id_client","id_site"])["pages_vues"].transform('sum')
dataframe['pourcentage_pages_vues_par_site'] = dataframe["somme_pages_vues_par_site"]/dataframe["somme_pages_vues_totale"]*100

#dummies
dataframe["id_site2"] = dataframe.id_site
dataframe = pd.get_dummies(dataframe, columns =['id_site'], prefix ='%visite_site')
dataframe.drop(["%visite_site_555493","%visite_site_40086","%visite_site_498703","%visite_site_413580",
                '%visite_site_499304'],axis=1, inplace=True, errors='ingore')

dataframe = pd.get_dummies(dataframe, columns =['id_site2'], prefix ='%pages_vues_site')
dataframe.drop(["%pages_vues_site_555493","%pages_vues_site_40086","%pages_vues_site_498703","%pages_vues_site_413580",
                '%pages_vues_site_499304'],axis=1, inplace=True, errors='ingore')


#mutliplication des dummies pour optenir le pourcentage de visites par site
dataframe[['%visite_site_492987','%visite_site_496306','%visite_site_496307','%visite_site_496838','%visite_site_502193'
,'%visite_site_509043','%visite_site_539121','%visite_site_548647']] = dataframe[['%visite_site_492987','%visite_site_496306'
,'%visite_site_496307','%visite_site_496838','%visite_site_502193','%visite_site_509043','%visite_site_539121'
,'%visite_site_548647']].multiply(dataframe["pourcentage_visites_par_site"], axis="index")

#mutliplication des dummies pour optenir le pourcentage de pages vues par site
dataframe[['%pages_vues_site_492987','%pages_vues_site_496306','%pages_vues_site_496307','%pages_vues_site_496838'
,'%pages_vues_site_502193','%pages_vues_site_509043','%pages_vues_site_539121'
,'%pages_vues_site_548647']] = dataframe[['%pages_vues_site_492987','%pages_vues_site_496306','%pages_vues_site_496307'
,'%pages_vues_site_496838','%pages_vues_site_502193','%pages_vues_site_509043','%pages_vues_site_539121'
,'%pages_vues_site_548647']].multiply(dataframe["pourcentage_pages_vues_par_site"], axis="index")

dataframe[['%visite_site_492987','%visite_site_496306','%visite_site_496307','%visite_site_496838','%visite_site_502193'
,'%visite_site_509043','%visite_site_539121','%visite_site_548647']] = dataframe.groupby(["id_client"])['%visite_site_492987'
,'%visite_site_496306','%visite_site_496307','%visite_site_496838','%visite_site_502193','%visite_site_509043'
,'%visite_site_539121','%visite_site_548647'].transform('max')

dataframe[['%pages_vues_site_492987','%pages_vues_site_496306','%pages_vues_site_496307','%pages_vues_site_496838'
,'%pages_vues_site_502193','%pages_vues_site_509043','%pages_vues_site_539121'
,'%pages_vues_site_548647']] = dataframe.groupby(["id_client"])['%pages_vues_site_492987','%pages_vues_site_496306'
,'%pages_vues_site_496307','%pages_vues_site_496838','%pages_vues_site_502193','%pages_vues_site_509043'
,'%pages_vues_site_539121','%pages_vues_site_548647'].transform("max")


dataframe.drop(['somme_visites_par_site', 'somme_pages_vues_par_site','premiere_visite', 'derniere_visite'
                , 'fileid', 'visites', 'pages_vues', 'pourcentage_pages_vues_par_site','pourcentage_visites_par_site']
               , axis=1, inplace=True)
dataframe = dataframe.drop_duplicates()

dataframe.to_pickle(path+"\s_dataframe_final_clients_mam_pas_forcmnt_juin.pkl")

#client ayant fait au moins une visite entre mars et mai 2015 et au moins une en juin 2015
dataframe = dataframe.loc[dataframe.recence_de_visite < 31]
dataframe.to_pickle(path+"\s_dataframe_final_clients.pkl")

#df_client_clean = pd.read_pickle(path+"\dataframes\s_dataframes_final_clients.pkl")

### Clean commmandes 

In [15]:
path = 'W:\L_equipe\\'
dataframe= pd.read_pickle(path+"\dataframes\s_dataframe_general_vel_commande.pkl")

def nettoyage_commande(dataframe, liste_client) :
    #selection des abonnés de l'equipe
    dataframe = dataframe.loc[dataframe['ClientUserId'].isin(liste_client)]
    #formatage datetime 
    dataframe['OrderDate'] = pd.to_datetime(dataframe['OrderDate'], format='%d/%m/%Y %H:%M:%S')

    #On ne considère pas les achats et les souscriptions post juin 2015
    dataframe = dataframe.loc[dataframe.OrderDate < date(2015,7,1)]
       
    dataframe = dataframe.loc[dataframe.OrderStatus == 'Completed']
    #Pour bien faire le lien avec la table souscription on réccupère les nouveaux abonnés 2015
    datasub = dataframe.loc[dataframe.ProductType == 'Service']
    datasub['anciennete_souscripteur'] = datasub.groupby(['ClientUserId']).OrderDate.transform('min')
    datasub = datasub.loc[((datasub.anciennete_souscripteur > date(2015,1,1)) & (datasub.OrderDate < date(2015,7,1))) ]
    l_sub = datasub.ClientUserId.unique()
    datasub = datasub[["ClientUserId", "anciennete_souscripteur"]]
    datasub = datasub.drop_duplicates()
    
    dataframe = dataframe.merge(datasub,how='left', left_on='ClientUserId', right_on='ClientUserId')
    
    
    #fusion du nettoyage des souscriptions avec les commandes
    dataframe.rename(columns={'ServiceId' : "ServiceID"}, inplace=True)
    df_sous = pd.read_excel(path+'\souscription_detail.xlsx')
    df_sous.drop(['ServiceTitle'], axis=1, inplace=True)
    dataframe = pd.merge(dataframe, df_sous, how='left', on="ServiceID")
        
    dataframe["sub_month"] = numpy.nan
    dataframe.loc[(dataframe.anciennete_souscripteur > date(2015,1,1)) 
                  & (dataframe.anciennete_souscripteur < date(2015,2,1)), 'sub_month'] = '1'
    dataframe.loc[(dataframe.anciennete_souscripteur > date(2015,2,1)) 
                  & (dataframe.anciennete_souscripteur < date(2015,3,1)), 'sub_month'] = '2'
    dataframe.loc[(dataframe.anciennete_souscripteur > date(2015,3,1)) 
                  & (dataframe.anciennete_souscripteur < date(2015,4,1)), 'sub_month'] = '3'
    dataframe.loc[(dataframe.anciennete_souscripteur > date(2015,4,1)) 
                  & (dataframe.anciennete_souscripteur < date(2015,5,1)), 'sub_month'] = '4'
    dataframe.loc[(dataframe.anciennete_souscripteur > date(2015,5,1)) 
                  & (dataframe.anciennete_souscripteur < date(2015,6,1)), 'sub_month'] = '5'
    dataframe.loc[(dataframe.anciennete_souscripteur > date(2015,6,1)) 
                  & (dataframe.anciennete_souscripteur < date(2015,7,1)), 'sub_month'] = '6' 
    
    #group by
    
    dataframe['recence_souscripteur'] = 0
    dataframe['recence_souscripteur'].loc[dataframe.ProductType == 'Service'] =\
    dataframe.loc[dataframe.ProductType == 'Service'].groupby(['ClientUserId'])["OrderDate"].transform('max')
    dataframe["recence_souscripteur"].loc[dataframe.ProductType == 'Service'] = (date(2015, 7, 1) - dataframe.recence_souscripteur)
    dataframe.recence_souscripteur = pd.to_numeric(dataframe.recence_souscripteur)/(1000000000 * 60 * 60 * 24)
    
    dataframe.anciennete_souscripteur = (date(2015, 7, 1) - dataframe.anciennete_souscripteur)
    dataframe.anciennete_souscripteur = pd.to_numeric(dataframe.anciennete_souscripteur)/(1000000000 * 60 * 60 * 24)
    
    #On ne prend en compte que les commande pré souscription
    dataframe["article_OK"] = 0
    dataframe.loc[dataframe.anciennete_souscripteur == 0 , "article_OK"] = 1
    dataframe["numeric_orderdate"] = (date(2015, 7, 1) - dataframe.OrderDate)
    dataframe.numeric_orderdate = pd.to_numeric(dataframe.numeric_orderdate)/(1000000000 * 60 * 60 * 24)
    dataframe.loc[((dataframe.ProductType == "Digital") & (dataframe.numeric_orderdate >dataframe.anciennete_souscripteur)), "article_OK"] = 1
                                                           
    dataframe.loc[((dataframe.ProductType == "Service") | (dataframe.article_OK == 1))]
    

    dataframe["recence_article"] = 0
    dataframe['recence_article'].loc[dataframe.ProductType == 'Digital'] =\
    dataframe.loc[dataframe.ProductType == 'Digital'].groupby(['ClientUserId'])["OrderDate"].transform('max')
    dataframe["recence_article"].loc[dataframe.ProductType == 'Digital'] = (date(2015, 7, 1) - dataframe.recence_article)
    dataframe.recence_article = pd.to_numeric(dataframe.recence_article)/(1000000000 * 60 * 60 * 24)

    dataframe["anciennete_article"] = 0
    dataframe['anciennete_article'].loc[dataframe.ProductType == "Digital"] =\
    dataframe.loc[dataframe.ProductType == "Digital"].groupby(['ClientUserId'])["OrderDate"].transform('min')
    dataframe.anciennete_article.loc[dataframe.ProductType == "Digital"] = (date(2015, 7, 1) - dataframe.anciennete_article)
    dataframe.anciennete_article = pd.to_numeric(dataframe.anciennete_article)/(1000000000 * 60 * 60 * 24)
    
    dataframe["somme_paiement_par_abonnement"] =0
    dataframe["somme_paiement_par_abonnement"].loc[dataframe.ProductType == 'Service'] = \
    dataframe.loc[dataframe.ProductType == 'Service'].groupby(["ClientUserId","ServiceID"]).GrossAmount.transform('sum')
    
    dataframe['nb_mensualites'] = 0
    dataframe['nb_mensualites'].loc[((dataframe.ProductType == 'Service') & (dataframe.duree_abonnement_theorique ==0 ))] =\
    dataframe.loc[dataframe.ProductType == 'Service'].groupby(["ClientUserId","ServiceID"]).BasketNumber.transform('count')
     

    dataframe["somme_mensualites"] = 0
    dataframe["somme_mensualites"].loc[((dataframe.ProductType == 'Service') & (dataframe.duree_abonnement_theorique ==0 ))] =\
    dataframe.loc[((dataframe.ProductType == 'Service') & (dataframe.duree_abonnement_theorique ==0 ))]\
    .groupby(["ClientUserId"]).BasketNumber.transform('count')
    
    
    dataframe["mensualite_moyenne"] = 0
    dataframe["mensualite_moyenne"].loc[((dataframe.ProductType == 'Service') & (dataframe.duree_abonnement_theorique ==0 ))] =\
    dataframe.loc[((dataframe.ProductType == 'Service') & (dataframe.duree_abonnement_theorique ==0 ))]\
    .groupby(['ClientUserId']).GrossAmount.transform("sum")
    dataframe["mensualite_moyenne"] = dataframe.mensualite_moyenne / dataframe.somme_mensualites
   
    
    #pour les abos fermes :
    dataframe.loc[((dataframe.somme_mensualites == 0) & (dataframe.duree_abonnement_theorique > 0)), "somme_mensualites"] =\
    dataframe.loc[((dataframe.somme_mensualites == 0) & (dataframe.duree_abonnement_theorique > 0))].duree_abonnement_theorique/30
    
    dataframe.loc[((dataframe.somme_mensualites == 0) & (dataframe.duree_abonnement_theorique > 0)), "mensualite_moyenne"] =\
    dataframe.loc[((dataframe.somme_mensualites == 0) & (dataframe.duree_abonnement_theorique > 0))].GrossAmount/\
    dataframe.loc[((dataframe.somme_mensualites == 0) & (dataframe.duree_abonnement_theorique > 0))].somme_mensualites
   
    dataframe['somme_paiement_totale'] = dataframe.groupby(["ClientUserId"]).GrossAmount.transform('sum')
    
    dataframe["somme_paiement_article"] =0
    dataframe["somme_paiement_article"].loc[dataframe.ProductType == 'Digital'] = \
    dataframe.loc[dataframe.ProductType == 'Digital'].groupby(["ClientUserId"]).GrossAmount.transform('sum')
        
    #features
    dataframe.ServiceID.fillna(0, inplace=True)
    #dataframe['theme_article'] = dataframe['ARTICLE_URL'].str.extract(('.*www.lequipe.fr/([a-zA-Z]+)/.*'))
    #dataframe.loc[dataframe.somme_paiement_par_abonnement == 0, 'commande_payante']=1
    #dataframe.loc[dataframe.somme_paiement_par_abonnement != 0, "commande_payante"]=0
    
    
    #dummies
    dataframe = pd.get_dummies(dataframe, columns = ['ProductType'], prefix ='prod_type')
    dataframe = pd.get_dummies(dataframe , columns=['sub_month'], prefix = 'sub_month')
    
    #dataframe['theme_article'] = dataframe['ARTICLE_URL'].str.extract(('.*www.lequipe.fr/([a-zA-Z]+)/.*'))
    #dataframe = pd.get_dummies(dataframe, columns=['theme_article'], prefix='theme_article')
    
    #duplication des infos utiles pour split abonnements et articles
    
    
    dataframe['nb_articles'] = dataframe.prod_type_Digital
    dataframe['nb_articles'] = dataframe.groupby(['ClientUserId']).nb_articles.transform("sum")
   
    #drop
    dataframe.drop(["AffiliateId","BasketNumber","Country","County","Currency","District","NetAmount","HouseFlatNumber",
        "HouseName","Message","Name","OfferId","OrderStatus","ParameterOrderId","PaymentMethod","PostCode","ProductDataSource",
        "Street","SupplierID","ThirdPartyRef","Titre","TotalTaxPercentage","TownCity","VATAmount","detail_mode_paiement","gift_id",
        "identifiant_commercial","identifiant_salon","lien","provenance","type",'ActivationCode',
        'DeliveryComments', 'subscriptionId','ContentItemId', "ARTICLE_TITLE", "ARTICLE_URL"
       ,'GrossAmount', 'OrderID', 'AccountID', "MPPGiftCode","Description", 'MPPGiftID', 'OrderDate', 'somme_paiement_par_abonnement',
         'ServiceID', 'article_OK',"numeric_orderdate"], axis=1, inplace=True)
    
    #on ne garde pas les souscription de avril/mai
    dataframe = dataframe.loc[((dataframe.sub_month_4==0) & (dataframe.sub_month_5==0))]
    dataframe.drop(['sub_month_4','sub_month_5','sub_month_6'], axis=1, inplace=True)
    dataframe = dataframe.drop_duplicates()
  
    dataframe[["duree_abonnement_theorique","prix_mensuel","occasion","cadeau","prix_mensuel_cadeau","duree_cadeau",
              "prod_type_Digital","prod_type_Service","anciennete_souscripteur"
              ,"anciennete_article","mensualite_moyenne", "nb_mensualites"]] = dataframe.groupby(["ClientUserId"])["duree_abonnement_theorique"
            ,"prix_mensuel","occasion","cadeau","prix_mensuel_cadeau","duree_cadeau","prod_type_Digital","prod_type_Service"
            ,"anciennete_souscripteur","anciennete_article","mensualite_moyenne","nb_mensualites"].transform(max)
    #print dataframe.loc[dataframe.prod_type_Digital == 1].prod_type_Service.value_counts()
    #print dataframe.loc[dataframe.prod_type_Digital == 0].prod_type_Service.value_counts()
    #return

    dataframe[['recence_article', 'recence_souscripteur']] = dataframe.groupby(['ClientUserId'])['recence_article'
                    , 'recence_souscripteur'].transform('min')
    
    dataframe = dataframe.drop_duplicates()
    
    dataframe["nb_abonnements"] = dataframe.groupby(["ClientUserId"]).prod_type_Service.transform("sum")
    dataframe[["somme_mensualites", "somme_paiement_article"]] = dataframe.groupby(["ClientUserId"])["somme_mensualites"
    ,"somme_paiement_article"].transform(sum)

    dataframe = dataframe.drop_duplicates()
    dataframe.fillna(0, inplace=True)
    return dataframe

df_commande = nettoyage_commande(dataframe, liste_client)
#df_commande.to_pickle(path+"\dataframes\s_dataframe_final_commandes.pkl")


#df_commande.sort_values(by='ClientUserId')
#df_commande.nb_abonnements.value_counts(dropna=False)
#pandas_profiling.ProfileReport(df_commande)

0.0    6540
1.0     135
Name: prod_type_Service, dtype: int64
1.0    2230
Name: prod_type_Service, dtype: int64


### Clean Session

In [3]:
# feature pour indiquer à partir de quelle date on regarde pour les sub 
path = "W:\L_equipe\dataframes\\"
sub = pd.read_pickle(path+"\s_dataframe_final_sub_payant_juin_2015.pkl")
sub["date_observation_min"] = sub.SubscriptionCreated - pd.DateOffset(months=3)
#dateutil.relativedelta.relativedelta(months=3)
sub["abo_juin"] = 1
sub.to_pickle(path+"\s_dataframe_final_sub_payant_juin_2015.pkl")

abo_2015 = pd.read_pickle(path+"\s_dataframe_final_sub_2015.pkl")
abo_2015["subscriber"] = 1
abo_2015.to_pickle(path+"\s_dataframe_final_sub_2015.pkl")

In [8]:
#dataframe= pd.read_pickle(path+"\dataframes\s_dataframe_general_xiti_sessions_mars_juin2015.pkl")
path = "W:\L_equipe\dataframes\\"
files  = []
files.extend([join(path,f) for f in listdir(path) if isfile(join(path, f)) 
                        if f.startswith('s_dataframe_general_xiti_sessions') if getsize(join(path,f))>0])

for f in files :
    dataframe = pd.read_pickle(f)
    reg = re.match(('.*s_dataframe_general_xiti_sessions_(.*)\.pkl'), f)
    
    #merge avec  les sub juin
    sub = pd.read_pickle(path+"\s_dataframe_final_sub_payant_juin_2015.pkl")
    sub2 = sub[["ClientUserId", "date_observation_min", "abo_juin"]]
    dataframe = pd.merge(dataframe, sub2, how='left', left_on='id_client', right_on="ClientUserId")
    dataframe.abo_juin.fillna(0, inplace=True)
    date_min_visiteur = date(2015,3,1)
    dataframe.date_observation_min.fillna(date_min_visiteur,inplace=True)
    dataframe["date_observation_max"] = dataframe.date_observation_min + pd.DateOffset(months = 3) - pd.DateOffset(days = 1)
    dataframe.loc[((dataframe.debut_de_session > dataframe.date_observation_min) & (dataframe.debut_de_session < dataframe.date_observation_max))]
    
    #merge avec les sub 2015 pour ne garder que les non subs ou les sub juin 2015
    abo_2015 = pd.read_pickle(path+"\s_dataframe_final_sub_2015.pkl")
    abo2 = abo_2015[["ClientUserId", 'subscriber']]
    dataframe = pd.merge(dataframe, abo2, how='left', left_on='id_client', right_on="ClientUserId")
    dataframe.subscriber.fillna(0, inplace=True)
    dataframe = dataframe.loc[((dataframe.subscriber == 0) | (dataframe.abo_juin==1))]

    #def clean_session(dataframe, liste_client) :
    #selection des abonnés de l'equipe
    dataframe = dataframe.loc[dataframe['id_client'].isin(liste_client)]
    #suppression des lignes ou fin de session = u'0001-01-01 00:00:00'
    dataframe = dataframe.loc[dataframe.fin_de_session != u'0001-01-01 00:00:00']
    #formatage datetime 
    dataframe['debut_de_session'] = pd.to_datetime(dataframe['debut_de_session'], format='%Y-%m-%d %H:%M:%S')
    dataframe['fin_de_session'] = pd.to_datetime(dataframe['fin_de_session'], format='%Y-%m-%d %H:%M:%S')
    dataframe['duree_session'] = dataframe.fin_de_session - dataframe.debut_de_session
    
    dataframe.loc[((dataframe.debut_de_session.dt.hour >= 0) & (dataframe.debut_de_session.dt.hour <= 5)), "periode" ] = "nuit"
    dataframe.loc[((dataframe.debut_de_session.dt.hour >= 6) & (dataframe.debut_de_session.dt.hour <=8)), "periode" ] = "aube"
    dataframe.loc[((dataframe.debut_de_session.dt.hour >= 9) & (dataframe.debut_de_session.dt.hour <12)), "periode" ] = "matinee"
    dataframe.loc[((dataframe.debut_de_session.dt.hour >= 12) & (dataframe.debut_de_session.dt.hour < 14)), "periode" ] = "midi"
    dataframe.loc[((dataframe.debut_de_session.dt.hour >= 14) & (dataframe.debut_de_session.dt.hour < 17)), "periode"] = "apres_midi"
    dataframe.loc[((dataframe.debut_de_session.dt.hour >=17 )& (dataframe.debut_de_session.dt.hour <20)), 'periode'] = 'fin_apres_midi'
    dataframe.loc[((dataframe.debut_de_session.dt.hour >= 20 ) & (dataframe.debut_de_session.dt.hour < 24)), "periode"] = "soiree"
    
    
    dataframe.loc[dataframe['localisation_3eme_niveau'] == 'Paris', 'Paris'] = 1
    dataframe.loc[dataframe['localisation_3eme_niveau'] != 'Paris', 'Paris'] = 0
    
    dataframe.loc[dataframe['localisation_2eme_niveau'] == 'Ile-de-France', 'RP'] = 1
    dataframe.loc[dataframe['localisation_2eme_niveau'] != 'Ile-de-France', 'RP'] = 0
    
    dataframe.loc[dataframe['localisation_1er_niveau'] == 'France', 'France'] = 1
    dataframe.loc[dataframe['localisation_1er_niveau'] != 'France', 'France'] = 0
    
    liste_semaines = ["semaine1","semaine2","semaine3","semaine4","semaine5","semaine6","semaine7","semaine8","semaine9","semaine10",
             "semaine11","semaine12"]
    liste_mois = ["mois1","mois2","mois3"]
    
    dataframe.loc[(dataframe.debut_de_session >= dataframe.date_observation_max - pd.DateOffset(months = 1)), "mois3"] = 1
    dataframe.loc[((dataframe.debut_de_session >= dataframe.date_observation_max - pd.DateOffset(months = 2)) &\
                   (dataframe.debut_de_session < dataframe.date_observation_max -pd.DateOffset(months =1))) , "mois2"] = 1
    dataframe.loc[((dataframe.debut_de_session >= dataframe.date_observation_max - pd.DateOffset(months = 3)) &\
                   (dataframe.debut_de_session < dataframe.date_observation_max -pd.DateOffset(months =2))), "mois1"] = 1
    
    dataframe.loc[(dataframe['debut_de_session'] >=dataframe.date_observation_max - pd.DateOffset(weeks = 1)), "semaine12" ] = 1
    dataframe.loc[((dataframe['debut_de_session'] >=dataframe.date_observation_max - pd.DateOffset(weeks = 2)) &\
                   (dataframe['debut_de_session'] < dataframe.date_observation_max - pd.DateOffset(weeks = 1))) , "semaine11" ] = 1
    dataframe.loc[((dataframe['debut_de_session'] >=dataframe.date_observation_max - pd.DateOffset(weeks = 3)) &\
                   (dataframe['debut_de_session'] < dataframe.date_observation_max - pd.DateOffset(weeks = 2))) , "semaine10" ] = 1
    dataframe.loc[((dataframe['debut_de_session'] >=dataframe.date_observation_max - pd.DateOffset(weeks = 4)) &\
                   (dataframe['debut_de_session'] < dataframe.date_observation_max - pd.DateOffset(weeks = 3))) , "semaine9" ] = 1
    dataframe.loc[((dataframe['debut_de_session'] >=dataframe.date_observation_max - pd.DateOffset(weeks = 5)) &\
                   (dataframe['debut_de_session'] < dataframe.date_observation_max - pd.DateOffset(weeks = 4))) , "semaine8" ] = 1
    dataframe.loc[((dataframe['debut_de_session'] >=dataframe.date_observation_max - pd.DateOffset(weeks = 6)) &\
                   (dataframe['debut_de_session'] < dataframe.date_observation_max - pd.DateOffset(weeks = 5))) , "semaine7" ] = 1
    dataframe.loc[((dataframe['debut_de_session'] >=dataframe.date_observation_max - pd.DateOffset(weeks = 7)) &\
                   (dataframe['debut_de_session'] < dataframe.date_observation_max - pd.DateOffset(weeks = 6))) , "semaine6" ] = 1
    dataframe.loc[((dataframe['debut_de_session'] >=dataframe.date_observation_max - pd.DateOffset(weeks = 8)) &\
                   (dataframe['debut_de_session'] < dataframe.date_observation_max - pd.DateOffset(weeks = 7))) , "semaine5" ] = 1
    dataframe.loc[((dataframe['debut_de_session'] >=dataframe.date_observation_max - pd.DateOffset(weeks = 9)) &\
                   (dataframe['debut_de_session'] < dataframe.date_observation_max - pd.DateOffset(weeks = 8))) , "semaine4" ] = 1
    dataframe.loc[((dataframe['debut_de_session'] >=dataframe.date_observation_max - pd.DateOffset(weeks = 10)) &\
                   (dataframe['debut_de_session'] < dataframe.date_observation_max - pd.DateOffset(weeks = 9))) , "semaine3" ] = 1
    dataframe.loc[((dataframe['debut_de_session'] >=dataframe.date_observation_max - pd.DateOffset(weeks = 11)) &\
                   (dataframe['debut_de_session'] < dataframe.date_observation_max - pd.DateOffset(weeks = 10))) , "semaine2" ] = 1
    dataframe.loc[((dataframe['debut_de_session'] >=dataframe.date_observation_max - pd.DateOffset(weeks = 12)) &\
                   (dataframe['debut_de_session'] < dataframe.date_observation_max - pd.DateOffset(weeks = 11))) , "semaine1" ] = 1

    #dataframe.loc[((dataframe['debut_de_session'] <= date(2015, 6, 7)) &
    #              (dataframe.debut_de_session > date(2015,5,31))), 'juin_semaine_1'] = 1
    #dataframe.loc[(dataframe['debut_de_session'] <= date(2015, 6, 14))& 
    #              (dataframe['debut_de_session'] > date(2015, 6, 7)), 'juin_semaine_2'] = 1
    #dataframe.loc[(dataframe['debut_de_session'] <= date(2015, 6, 21) ) & 
    #              (dataframe['debut_de_session'] > date(2015, 6, 14)), 'juin_semaine_3'] = 1
    #dataframe.loc[(dataframe['debut_de_session'] > date(2015, 6, 21)) & 
    #              (dataframe.debut_de_session < date(2015,7,1)), 'juin_semaine_4'] = 1
    #
    #dataframe.loc[((dataframe['debut_de_session'] <= date(2015, 5,10)) &
    #              (dataframe.debut_de_session > date(2015,4,30))), 'mai_semaine_1'] = 1
    #dataframe.loc[(dataframe['debut_de_session'] <= date(2015, 5, 17))& 
    #              (dataframe['debut_de_session'] > date(2015, 5, 10)), 'mai_semaine_2'] = 1
    #dataframe.loc[(dataframe['debut_de_session'] <= date(2015, 5, 24) ) & 
    #              (dataframe['debut_de_session'] > date(2015, 5, 17)), 'mai_semaine_3'] = 1
    #dataframe.loc[(dataframe['debut_de_session'] > date(2015, 5, 24)) & 
    #              (dataframe.debut_de_session < date(2015,6,1)), 'mai_semaine_4'] = 1
    #                  
    #dataframe.loc[((dataframe['debut_de_session'] <= date(2015, 4, 5)) &
    #              (dataframe.debut_de_session > date(2015,3,31))), 'avril_semaine_1'] = 1
    #dataframe.loc[(dataframe['debut_de_session'] <= date(2015, 4, 12))& 
    #              (dataframe['debut_de_session'] > date(2015, 4, 5)), 'avril_semaine_2'] = 1
    #dataframe.loc[(dataframe['debut_de_session'] <= date(2015, 4, 19) ) & 
    #              (dataframe['debut_de_session'] > date(2015, 4, 12)), 'avril_semaine_3'] = 1
    #dataframe.loc[(dataframe['debut_de_session'] > date(2015, 4, 19)) & 
    #              (dataframe.debut_de_session < date(2015,5,1)), 'avril_semaine_4'] = 1
    #               
    #dataframe.loc[((dataframe['debut_de_session'] <= date(2015, 3, 8)) &
    #              (dataframe.debut_de_session > date(2015,2,28))), 'mars_semaine_1'] = 1
    #dataframe.loc[(dataframe['debut_de_session'] <= date(2015, 3, 15))& 
    #              (dataframe['debut_de_session'] > date(2015, 3, 8)), 'mars_semaine_2'] = 1
    #dataframe.loc[(dataframe['debut_de_session'] <= date(2015, 3, 22) ) & 
    #              (dataframe['debut_de_session'] > date(2015, 3, 15)), 'mars_semaine_3'] = 1
    #dataframe.loc[(dataframe['debut_de_session'] > date(2015, 3, 22)) & 
    #              (dataframe.debut_de_session < date(2015,4,1)), 'mars_semaine_4'] = 1
    
    liste_weekend = [date(2015,3,1),date(2015,3,7),date(2015,3,8),date(2015,3,14),date(2015,3,15), date(2015,3,21),date(2015,3,22),date(2015,3,28),
                     date(2015,3,29),date(2015,4,4),date(2015,4,5),date(2015,4,11),date(2015,4,12),date(2015,4,18),date(2015,4,19),date(2015,4,25),
                     date(2015,4,26),date(2015,5,2),date(2015,5,3),date(2015,5,9),date(2015,5,10),date(2015,5,16),date(2015,5,17),date(2015,5,23),
                     date(2015,5,24),date(2015,5,30),date(2015,5,31),date(2015,6,6),date(2015,6,7),date(2015,6,13),date(2015,6,14),date(2015,6,20),
                     date(2015,6,21),date(2015,6,27),date(2015,6,28)]
    
    dataframe["type_jour"] = "semaine"
    dataframe.loc[dataframe.debut_de_session.dt.date.isin(liste_weekend), "type_jour"] = "weekend"
    
    #liste_semaines = [col for col in list(dataframe) if col.find("semaine")]

    dataframe[liste_semaines] = dataframe.groupby(["id_client"])[liste_semaines].transform(max)
    dataframe[liste_mois] = dataframe.groupby(["id_client"])[liste_mois].transform(max)
    
    #dataframe["mars_semaine_1"] = dataframe.groupby(["id_client"])["mars_semaine_1"].transform("max")
    #dataframe["mars_semaine_2"] = dataframe.groupby(["id_client"])["mars_semaine_2"].transform("max")
    #dataframe["mars_semaine_3"] = dataframe.groupby(["id_client"])["mars_semaine_3"].transform("max")
    #dataframe["mars_semaine_4"] = dataframe.groupby(["id_client"])["mars_semaine_4"].transform("max")
    #dataframe["avril_semaine_1"] = dataframe.groupby(["id_client"])["avril_semaine_1"].transform("max")
    #dataframe["avril_semaine_2"] = dataframe.groupby(["id_client"])["avril_semaine_2"].transform("max")
    #dataframe["avril_semaine_3"] = dataframe.groupby(["id_client"])["avril_semaine_3"].transform("max")
    #dataframe["avril_semaine_4"] = dataframe.groupby(["id_client"])["avril_semaine_4"].transform("max")
    #dataframe["mai_semaine_1"] = dataframe.groupby(["id_client"])["mai_semaine_1"].transform("max")
    #dataframe["mai_semaine_2"] = dataframe.groupby(["id_client"])["mai_semaine_2"].transform("max")
    #dataframe["mai_semaine_3"] = dataframe.groupby(["id_client"])["mai_semaine_3"].transform("max")
    #dataframe["mai_semaine_4"] = dataframe.groupby(["id_client"])["mai_semaine_4"].transform("max")
    #dataframe["juin_semaine_1"] = dataframe.groupby(["id_client"])["juin_semaine_1"].transform("max")
    #dataframe["juin_semaine_2"] = dataframe.groupby(["id_client"])["juin_semaine_2"].transform("max")
    #dataframe["juin_semaine_3"] = dataframe.groupby(["id_client"])["juin_semaine_3"].transform("max")
    #dataframe["juin_semaine_4"] = dataframe.groupby(["id_client"])["juin_semaine_4"].transform("max")
    
    
    dataframe["somme_session"] = dataframe.groupby(["id_client"]).id_session.transform('count')
    dataframe = pd.get_dummies(dataframe, columns=["type_jour"])
    dataframe["somme_session_semaine"] = dataframe.groupby(["id_client"]).type_jour_semaine.transform('sum')
    dataframe["somme_session_weekend"] = dataframe.groupby(["id_client"]).type_jour_weekend.transform('sum')
    #dataframe["duree_session_moyenne"] = dataframe.groupby(["id_client"]).duree_session.transform("mean")
    dataframe["duree_session_min"] = dataframe.groupby(["id_client"]).duree_session.transform("min")
    dataframe["duree_session_max"] = dataframe.groupby(["id_client"]).duree_session.transform("max")
    dataframe["duree_session_totale"] = dataframe.groupby(["id_client"]).duree_session.transform("sum")
    
    dataframe = pd.get_dummies(dataframe, columns=["id_site"], prefix = "site")
    
    #liste_site = ["site_492987","site_496306","site_496307","site_496838","site_502193","site_509043","site_539121","site_548647"]
    
    dataframe["site_492987"] = dataframe.groupby(["id_client"])["site_492987"].transform("max")
    dataframe["site_496306"] = dataframe.groupby(["id_client"])["site_496306"].transform("max")
    dataframe["site_496307"] = dataframe.groupby(["id_client"])["site_496307"].transform("max")
    dataframe["site_496838"] = dataframe.groupby(["id_client"])["site_496838"].transform("max")
    dataframe["site_502193"] = dataframe.groupby(["id_client"])["site_502193"].transform("max")
    dataframe["site_509043"] = dataframe.groupby(["id_client"])["site_509043"].transform("max")
    dataframe["site_539121"] = dataframe.groupby(["id_client"])["site_539121"].transform("max")
    dataframe["site_548647"] = dataframe.groupby(["id_client"])["site_548647"].transform("max")
    
    
    dataframe["somme_pages_vues"] = dataframe.groupby(["id_client"]).pages_vues.transform("sum")
    #OS managing
    dataframe["os2"] = "autres"
    dataframe.os = dataframe.os.str.lower()
    dataframe.loc[dataframe.os.str.match(".*ios.*"), "os2"] = "iphone"
    dataframe.loc[dataframe.os.str.match(".*ipad.*"), "os2"] = "ipad"
    dataframe.loc[((dataframe.os.str.match(".*mac os.*")) | (dataframe.os2.str.match(".*os x.*")))  , "os2"] = "mac"
    dataframe.loc[dataframe.os.str.match(".*windows.*"), "os2"] = "windows"
    dataframe.loc[dataframe.os.str.match(".*windows phone.*"), "os2"] = "windows_phone"
    dataframe.loc[dataframe.os.str.match(".*android.*"), "os2"] = "android"
    dataframe.loc[dataframe.os.str.match(".*linux.*"), "os2"] = "linux"
    dataframe.loc[dataframe.os.str.match(".*chrome.*"), "os2"] = "chrome_os"
    
    dataframe = pd.get_dummies(dataframe, columns=["os2"])
    dataframe = pd.get_dummies(dataframe, columns= ["periode"])
    
    dataframe[["os2_iphone","os2_ipad","os2_mac","os2_windows","os2_windows_phone","os2_android","os2_linux",
              "os2_chrome_os", "os2_autres"]] = dataframe.groupby(["id_client"])["os2_iphone","os2_ipad","os2_mac","os2_windows"
                                ,"os2_windows_phone","os2_android","os2_linux","os2_chrome_os","os2_autres"].transform("max")
    
    dataframe[["periode_nuit","periode_aube","periode_matinee","periode_midi","periode_apres_midi",
               "periode_fin_apres_midi","periode_soiree"]] = dataframe.groupby(['id_client'])["periode_nuit",
            "periode_aube","periode_matinee","periode_midi","periode_apres_midi","periode_fin_apres_midi","periode_soiree"].transform('max')
    
    dataframe.drop(["fileid","id_campagne","id_session", "debut_de_session","fin_de_session", "pages_vues", 'localisation_1er_niveau',
                    'localisation_2eme_niveau', 'localisation_3eme_niveau', "terminal", "os", "duree_session","ClientUserId_y","ClientUserId_x"],axis=1, inplace=True)
    
    dataframe = dataframe.drop_duplicates()
    #return dataframe
    
    #df_session = clean_session(dataframe, liste_client)
    dataframe.to_pickle(path+"\s_dataframe_final_sessions_"+reg.group(1)+".pkl")


### on rassemble les différents fichiers

In [ ]:
#dataframe= pd.read_pickle(path+"\dataframes\s_dataframe_general_xiti_sessions_mars_juin2015.pkl")
path = "W:\L_equipe\dataframes\\"
final = pd.DataFrame()
files  = []
files.extend([join(path,f) for f in listdir(path) if isfile(join(path, f)) 
                        if f.startswith('s_dataframe_final_sessions_') if getsize(join(path,f))>0])

liste_semaines = ["semaine1","semaine2","semaine3","semaine4","semaine5","semaine6","semaine7","semaine8","semaine9","semaine10",
             "semaine11","semaine12"]
liste_mois = ["mois1","mois2","mois3"]

final= pd.concat(( pd.read_pickle(f) for f in files))

#final.drop(["site_555493","site_40086","site_498703","site_413580",'site_499304'],axis=1, inplace=True)
final.os2_autres = final.groupby(['id_client']).os2_autres.transform(max)

final[["Paris", "RP", "France"]] = final.groupby(["id_client"])["Paris","RP",'France'].transform("max")
final[["os2_iphone","os2_ipad","os2_mac","os2_windows","os2_windows_phone","os2_android","os2_linux",
              "os2_chrome_os", "os2_autres"]] = final.groupby(["id_client"])["os2_iphone","os2_ipad","os2_mac","os2_windows"
                                ,"os2_windows_phone","os2_android","os2_linux","os2_chrome_os","os2_autres"].transform("max")
final[["periode_nuit","periode_aube","periode_matinee","periode_midi","periode_apres_midi",
               "periode_fin_apres_midi","periode_soiree"]] = final.groupby(['id_client'])["periode_nuit",
            "periode_aube","periode_matinee","periode_midi","periode_apres_midi","periode_fin_apres_midi","periode_soiree"].transform('max')

final["site_492987"] = final.groupby(["id_client"])["site_492987"].transform("max")
final["site_496306"] = final.groupby(["id_client"])["site_496306"].transform("max")
final["site_496307"] = final.groupby(["id_client"])["site_496307"].transform("max")
final["site_496838"] = final.groupby(["id_client"])["site_496838"].transform("max")
final["site_502193"] = final.groupby(["id_client"])["site_502193"].transform("max")
final["site_509043"] = final.groupby(["id_client"])["site_509043"].transform("max")
final["site_539121"] = final.groupby(["id_client"])["site_539121"].transform("max")
final["site_548647"] = final.groupby(["id_client"])["site_548647"].transform("max")
 
#final["site_40086"] = final.groupby(['id_client'])["site_40086"].transform('max')
#final["site_498703"] = final.groupby(['id_client'])["site_498703"].transform('max')
#final["site_413580"] = final.groupby(['id_client'])["site_413580"].transform('max')
#final['site_499304'] = final.groupby(['id_client'])["site_499304"].transform('max')

final["duree_session_max"] = final.groupby(["id_client"]).duree_session_max.transform("max")
final[liste_semaines] = final.groupby(["id_client"])[liste_semaines].transform(max)
final[liste_mois] = final.groupby(["id_client"])[liste_mois].transform(max)

final[['type_jour_semaine','type_jour_weekend']] = final.groupby(['id_client'])['type_jour_semaine','type_jour_weekend'].transform(max)
final["duree_session_min"] = final.groupby(["id_client"]).duree_session_min.transform("min")
final = final.drop_duplicates()
final['somme_pages_vues'] = final.groupby(['id_client'])['somme_pages_vues'].transform(sum)
final['somme_session'] = final.groupby(['id_client'])['somme_session'].transform(sum)
final['somme_session_semaine'] = final.groupby(['id_client'])['somme_session_semaine'].transform(sum)
final['somme_session_weekend'] = final.groupby(['id_client'])['somme_session_weekend'].transform(sum)
final['duree_session_totale'] =final.groupby(['id_client'])['duree_session_totale'].transform(sum)

final = final.drop_duplicates()

In [26]:
final.to_pickle(path+"\s_dataframe_final_sessions.pkl")

### Clean pages

In [2]:
#création de la liste de la popolation a sélectionner dans page et sessions
path = r"W:\L_Equipe\dataframes"
df = pd.read_pickle(path+"\s_dataframe_final_clients.pkl")
df1 = pd.read_pickle(path+"\s_dataframe_final_sub_payant_juin_2015.pkl")
liste_pop = df.id_client.unique().tolist()
liste_sub_juin = df1.ClientUserId.unique().tolist()
set_pop=set(liste_pop)
set_sub_juin = set(liste_sub_juin)
set_pop.update(set_sub_juin)  #liste d'id unique contenant notre population

In [3]:
#premiere étape clean chaque fichiers page et fichier session avec le cut sur notre population 
path = "W:\L_equipe\dataframes\\"
dataframe = pd.DataFrame()
files  = []
files.extend([join(path,f) for f in listdir(path) if isfile(join(path, f)) 
                        if f.startswith('s_dataframe_general_xiti_pages') if getsize(join(path,f))>0])

for f in files :
    reg = re.match(('.*pages_(.*)'), f)
    dataframe = pd.read_pickle(f)
    dataframe = dataframe.loc[dataframe.id_site.isin([492987,496306,496307,496838,502193,509043,539121,548647])]
    dataframe = dataframe.loc[dataframe.id_client.isin(set_pop)]
    dataframe.to_pickle(path+"s_dataframe_temp_pages_population_"+reg.group(1))

IOError: [Errno 2] No such file or directory: 'W:\\L_equipe\\dataframes\\s_dataframe_general_xiti_pages_avril2015_partie1.pkl'

In [9]:
#deuxieme étape (à faire car le cut n'est pas suffisant) on group by et on refera une étape de groupby par la suite
path = "W:\L_equipe\dataframes\\"
files  = []
files.extend([join(path,f) for f in listdir(path) if isfile(join(path, f)) 
                        if f.startswith('s_dataframe_temp_pages_population') if getsize(join(path,f))>0])

for f in files :
    reg = re.match(('.*pages_population_(.*)\.pkl'), f)
    dataframe = pd.read_pickle(f)
    
    dataframe = pd.get_dummies(dataframe, columns=["id_site"], prefix="id_site")
    #normalisation de niveau_2
    dataframe.niveau_2 = dataframe.niveau_2.str.lower()
    dataframe.niveau_2 = dataframe.niveau_2.str.normalize('NFKD')
    dataframe.niveau_2 = dataframe.niveau_2.str.encode('ASCII', 'ignore')
    type_sports = pd.read_csv(r"W:\L_Equipe\types_sports.csv",sep=";")
    dataframe = dataframe.merge(type_sports,how='left', left_on='niveau_2', right_on='niveau_2')
    #dummies
    dataframe = pd.get_dummies(dataframe, columns= ["theme"], prefix='theme')
    #groupby
    dataframe["nb_session"] = dataframe.groupby(["id_client"]).id_session.transform("count")
    
    dataframe["nb_pages_par_session"] = dataframe.groupby(["fileid", "id_client","id_session"]).position_de_la_page.transform("max")
    dataframe['nb_pages_moyen'] = dataframe.groupby(["id_client"]).nb_pages_par_session.transform('mean')
    dataframe['nb_pages_min'] = dataframe.groupby(["id_client"]).nb_pages_par_session.transform('min')
    dataframe['nb_pages_max'] = dataframe.groupby(["id_client"]).nb_pages_par_session.transform('max')
    dataframe['nb_pages_total'] = dataframe.groupby(["id_client"]).nb_pages_par_session.transform('sum')
    
    dataframe["duree_session"] = dataframe.groupby(["fileid", "id_client","id_session"]).temps_passe_sur_la_page.transform("sum")
    dataframe["duree_session_moyenne"] = dataframe.groupby(["id_client"]).duree_session.transform('mean')
    dataframe["duree_session_max"] = dataframe.groupby(["id_client"]).duree_session.transform('max')
    dataframe["duree_session_min"] = dataframe.groupby(["id_client"]).duree_session.transform('min')
    
    sites = [col for col in list(dataframe) if col.startswith("id_site")]

    themes = [col for col in list(dataframe) if col.startswith('theme')]
    
    dataframe[sites] = dataframe.groupby(["id_client"])[sites].transform("max")
    dataframe[themes] = dataframe.groupby(["id_client"])[themes].transform("max")
    
    dataframe.drop(["url", "fileid","position_de_la_page","temps_passe_sur_la_page", "niveau_2","id_session"
                    ,"nb_pages_par_session","duree_session","index"],axis=1, inplace=True)
    
    dataframe = dataframe.drop_duplicates()
    #dataframe = dataframe.rename(columns = lambda x : reg.group(1) +'_'+ x)
    dataframe.to_pickle(path+"\s_dataframe_temp2_pages_population_"+reg.group(1))
    

### final groupby pages

In [ ]:
path = "W:\L_equipe\dataframes\\"
files  = []
dataframe = pd.DataFrame()
files.extend([join(path,f) for f in listdir(path) if isfile(join(path, f)) 
                        if f.startswith('s_dataframe_temp2_pages_population') if getsize(join(path,f))>0])

dataframe = pd.concat((pd.read_pickle(f) for f in files))

In [19]:
dataframe.sort_values(by='id_client')
listemax = ["duree_session_max","id_site_492987","id_site_492987.0","id_site_496306","id_site_496306.0",
            "id_site_496307","id_site_496307.0","id_site_496838","id_site_496838.0","id_site_502193",
            "id_site_502193.0","id_site_509043","id_site_509043.0","id_site_539121","id_site_539121.0",
            "id_site_548647","id_site_548647.0","nb_pages_max","theme_athletisme","theme_auto-moto",
            "theme_autres","theme_basket","theme_blogs","theme_cyclisme","theme_e-boutiques","theme_football",
            "theme_general","theme_golf","theme_handball","theme_hippisme","theme_natation","theme_rugby",
            "theme_ski-glace","theme_souscription","theme_tennis","theme_vente en ligne","theme_video","theme_voile"]

listemin = ["nb_pages_min","duree_session_min"]
dataframe[listemin] = dataframe.groupby(['id_client'])[listemin].transform(min)
dataframe[listemax] = dataframe.groupby(["id_client"])[listemax].transform(max)

dataframe = dataframe.drop_duplicates()

dataframe.drop(["duree_session_moyenne","nb_pages_moyen"], axis=1, inplace=True)
dataframe.nb_pages_total = dataframe.groupby(['id_client']).nb_pages_total.transform(sum)
dataframe.nb_session = dataframe.groupby(['id_client']).nb_session.transform(sum)
dataframe = dataframe.drop_duplicates()
dataframe.to_pickle(path+"\s_dataframe_final_page.pkl")

In [19]:
commandes

,ClientUserId,anciennete_souscripteur,duree_abonnement_theorique,prix_mensuel,occasion,cadeau,prix_mensuel_cadeau,duree_cadeau,recence_souscripteur,recence_article,anciennete_article,nb_mensualites,somme_mensualites,mensualite_moyenne,somme_paiement_totale,somme_paiement_article,prod_type_Digital,prod_type_Service,nb_articles,nb_abonnements
0,1537877,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.006007,30.006007,0.0,0.0,0.0,0.49,0.49,1.0,0.0,1.0,0.0
3,1449417,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.052130,30.052130,0.0,0.0,0.0,0.49,0.49,1.0,0.0,1.0,0.0
6,1543488,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.069769,30.069769,0.0,0.0,0.0,0.99,0.99,1.0,0.0,1.0,0.0
7,1543485,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.076574,30.076574,0.0,0.0,0.0,0.49,0.49,1.0,0.0,1.0,0.0
9,335531,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.616042,35.091296,0.0,0.0,0.0,16.83,16.83,1.0,0.0,17.0,0.0
11,1543475,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.101562,30.101562,0.0,0.0,0.0,0.99,0.99,1.0,0.0,1.0,0.0
13,1152639,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.108808,54.074317,0.0,0.0,0.0,4.99,4.99,1.0,0.0,2.0,0.0
14,501133,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.109039,30.109039,0.0,0.0,0.0,0.99,0.99,1.0,0.0,1.0,0.0
15,624282,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.204595,67.394884,0.0,0.0,0.0,23.30,23.30,1.0,0.0,30.0,0.0
21,294666,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.126898,78.303576,0.0,0.0,0.0,13.99,13.99,1.0,0.0,11.0,0.0


### Concatenation finale

In [28]:
dataframe = pd.DataFrame()

path = "W:\L_equipe\dataframes\\"
client =pd.read_pickle(path+"\s_dataframe_final_clients_mam_pas_forcmnt_juin.pkl")
pages = pd.read_pickle(path+"\s_dataframe_final_page.pkl")
sessions = pd.read_pickle(path+"\s_dataframe_final_sessions.pkl")
commandes = pd.read_pickle(path+"\s_dataframe_final_commandes.pkl")
sub_juin = pd.read_pickle(path+"\s_dataframe_final_sub_payant_juin_2015.pkl")

In [29]:
dataframe = pages.merge(client,how='inner', left_on='id_client', right_on='id_client')
dataframe = dataframe.merge(sessions, how="inner", left_on='id_client', right_on='id_client')
dataframe = dataframe.merge(commandes, how="left", left_on='id_client', right_on='ClientUserId')

In [11]:
liste_client = client.id_client.unique()
liste_pages = pages.id_client.unique()
liste_sessions = sessions.id_client.unique()
#liste_commandes = commandes.ClientUserId.unique()
liste_sub_juin = sub_juin.ClientUserId.unique()

In [12]:
s=set(liste_sub_juin)
t=set(liste_pages)
u=set(liste_sessions)
#v=set(liste_commandes)
w=set(liste_client)
print "taille de l'intersection sub_juin sessions : ", len(s.intersection(u))
#print "taille de l'intersection sub_juin commandes : ", len(s.intersection(v))
print "taille de l'intersection sub_juin client : ", len(s.intersection(w))
print "taille de l'intersection sub_juin pages : ", len(s.intersection(t))

taille de l'intersection sub_juin sessions :  399
taille de l'intersection sub_juin client :  402
taille de l'intersection sub_juin pages :  402


In [30]:
#derniers drops et ajustements
dataframe.drop(["date_observation_max","date_observation_min"], axis=1, inplace=True)
dataframe.duree_session_totale = dataframe.duree_session_totale.dt.seconds

#drop for too high corelation
#dataframe.drop(["%pages_vues_site_492987","%pages_vues_site_496306","%pages_vues_site_496307","%pages_vues_site_496838",
#                "%pages_vues_site_502193","%pages_vues_site_509043","%pages_vues_site_539121","%pages_vues_site_548647",
#                "ClientUserId","cadeau","duree_session_max_y","duree_session_min_y","site_496306","site_496307",
#                "site_496838","site_502193","site_509043","site_539121","site_492987","site_548647","site_40086",
#                "site_413580","site_498703","site_499304","site_555493","somme_pages_vues",
#                "id_site_492987.0","id_site_496306.0","id_site_496307.0","id_site_496838.0","id_site_502193.0",
#                "id_site_509043.0","id_site_539121.0","id_site_548647.0"], axis=1, inplace=True)
#
dataframe.drop(["%pages_vues_site_492987","%pages_vues_site_496306","%pages_vues_site_496307","%pages_vues_site_496838",
                "%pages_vues_site_502193","%pages_vues_site_509043","%pages_vues_site_539121","%pages_vues_site_548647",
                "duree_session_max_y","duree_session_min_y","site_496306","site_496307",
                "site_496838","site_502193","site_509043","site_539121","site_492987","site_548647","site_40086",
                "site_413580","site_498703","site_499304","site_555493","somme_pages_vues",
                "id_site_492987.0","id_site_496306.0","id_site_496307.0","id_site_496838.0","id_site_502193.0",
                "id_site_509043.0","id_site_539121.0","id_site_548647.0"], axis=1, inplace=True)

#high corelation pas sur 
dataframe.drop(["somme_pages_vues_totale","somme_session","recence_souscripteur",
                "somme_mensualites","nb_abonnements"], axis=1, inplace=True)
#
#drop 
dataframe.drop(["anciennete_souscripteur","duree_abonnement_theorique","duree_cadeau","mensualite_moyenne",
               "nb_mensualites","occasion","prix_mensuel","prix_mensuel_cadeau","prod_type_Service",
                "somme_paiement_totale", "cadeau", "ClientUserId","anciennete_article","recence_article"], axis=1, inplace=True)

dataframe["multidevice"] = dataframe.id_site_492987 + dataframe.id_site_496306 + dataframe.id_site_496307\
                + dataframe.id_site_496838 + dataframe.id_site_502193 + dataframe.id_site_509043 + dataframe.id_site_539121\
                + dataframe.id_site_548647
#juste pour les semaines
dataframe.fillna(0, inplace=True)
dataframe = dataframe.drop_duplicates()
#dataframe.drop(["id_client"])

In [31]:
dataframe.to_pickle(path+"\dataframe_final_avec_commande.pkl")

### Random Undersampling

In [32]:
path = "W:\L_equipe\dataframes\\"
dataframe = pd.read_pickle(path+"\dataframe_final_avec_commande.pkl")
#dataframe = pd.pd.read_pickle(path+"\dataframe_final.pkl")
non_abo = dataframe.loc[dataframe.abo_juin == 0].sample(n = 300)
abo = dataframe.loc[dataframe.abo_juin == 1].sample(n=300)
reste = dataframe.loc[(~dataframe.id_client.isin(non_abo.id_client)& ~dataframe.id_client.isin(abo.id_client))]
reste_nonabo = reste.loc[reste.abo_juin == 0]
reste_abo = reste.loc[reste.abo_juin ==1]
framestest = [reste_nonabo, reste_abo]
frames = [non_abo,abo]
dataframe_final = pd.concat(frames)
dataframe_final_test = pd.concat(framestest)
dataframe_final.to_pickle(path+"\dataframe_avec_commande_ready_to_use_300_300.pkl")
print reste.shape
dataframe_final_test.to_pickle(path+"\dataframe_avec_commande_test_300_300_full_non_abo.pkl")

(122883, 91)


### Sample test

In [7]:
path = "W:\L_equipe\dataframes\\"
final = pd.read_pickle(path+"\dataframe_avec_commande_ready_to_use_300_300.pkl")
dataframe = dataframe = pd.read_pickle(path+"\dataframe_final_avec_commande.pkl")
reste = dataframe.loc[~dataframe.id_client.isin(final.id_client)]
reste_nonabo = reste.loc[reste.abo_juin == 0].sample(n=99)
reste_abo = reste.loc[reste.abo_juin ==1]
framestest = [reste_nonabo, reste_abo]
dataframe_final_test = pd.concat(framestest)
print dataframe_final_test.shape
dataframe_final_test.to_pickle(path+"\dataframe_avec_commande_test_300_300_sample_non_abo.pkl")

(198, 91)


### Notes, tests, ...

In [ ]:
test = dataframe.loc[dataframe.abo_juin == 0]
test = test.sample(n = 10000)
test.to_pickle(path+"test_pour_voir.pkl")

In [ ]:
#clean datetime
#df_commande_clean['OrderDate'] = pd.to_datetime(df_commande_clean['OrderDate'])
#df_compte_clean['LastUpdated'] = pd.to_datetime(df_compte_clean['LastUpdated'])
#df_compte_clean['CreateDate'] = pd.to_datetime(df_compte_clean['CreateDate'])


df_session_juin['debut_de_session'] = pd.to_datetime(df_session_juin['debut_de_session'])
df_session_juin['fin_de_session'] = pd.to_datetime(df_session_juin['fin_de_session'])


#df_client_clean.premiere_v = pd.to_datetime(df_client_clean['premiere_v'])
#df_client_clean.derniere_v = pd.to_datetime(df_client_clean['derniere_v'])

#
df_session_juin["temps_session"] =  df_session_juin["fin_de_session"] - df_session_juin["debut_de_session"]

In [ ]:
#création de différents indicateurs
df_session_juin.loc[df_session_juin['localisation_3eme_niveau'] == 'Paris', 'Paris'] = 1
df_session_juin.loc[df_session_juin['localisation_3eme_niveau'] != 'Paris', 'Paris'] = 0

df_session_juin.loc[df_session_juin['localisation_2eme_niveau'] == 'Ile-de-France', 'RP'] = 1
df_session_juin.loc[df_session_juin['localisation_2eme_niveau'] != 'Ile-de-France', 'RP'] = 0

df_session_juin.loc[df_session_juin['localisation_1er_niveau'] == 'France', 'France'] = 1
df_session_juin.loc[df_session_juin['localisation_1er_niveau'] != 'France', 'France'] = 0


df_session_juin.loc[(df_session_juin['debut_de_session'] < date(2015, 6, 7)), 'juin_semaine_1'] = 1
#df_session_juin_clean.loc[(df_session_juin_clean['debut_de_session'] > date(2015, 6, 7)), 'juin_semaine_1'] = 0

df_session_juin.loc[(df_session_juin['debut_de_session'] < date(2015, 6, 14) ) 
                          & (df_session_juin['debut_de_session'] > date(2015, 6, 6)), 'juin_semaine_2'] = 1
#df_session_juin_clean.loc[(df_session_juin_clean['debut_de_session'] < date(2015, 6, 7) ) 
#                          & (df_session_juin_clean['debut_de_session'] > date(2015, 5, 31)), 'juin_semaine_1'] = 1

df_session_juin.loc[(df_session_juin['debut_de_session'] < date(2015, 6, 21) ) 
                          & (df_session_juin['debut_de_session'] > date(2015, 6, 13)), 'juin_semaine_3'] = 1

df_session_juin.loc[(df_session_juin['debut_de_session'] > date(2015, 6, 20) ), 'juin_semaine_4'] = 1


In [ ]:
df_session_juin

In [ ]:
#travail sur les dates des clients : 
df_client_clean["anciennete"] = (date(2015, 6, 30) - df_client_clean["premiere_v"])
df_client_clean["derniere_visite"] = (date(2015, 6, 30) - df_client_clean["derniere_v"])
df_client_clean.anciennete = pd.to_numeric(df_client_clean.anciennete)/(1000000000 * 60 * 60 * 24)
df_client_clean.derniere_visite = pd.to_numeric(df_client_clean.derniere_visite)/(1000000000 * 60 * 60 * 24)

In [ ]:
plt.figure()
#df_session_juin["debut_de_session"].groupby(df_session_juin["debut_de_session"].dt.hour).count().plot(kind="bar")
df_session_juin["debut_de_session"].groupby(df_session_juin["debut_de_session"].dt.hour).count().plot(kind="bar")
plt.savefig("debut_de_session_juin2015.png")

In [ ]:
df_commande.rename(columns = {'Somme_NetAmmount':'Somme_NetAmount'}, inplace = True)

In [ ]:
print (df_pages_juin_p1.theme_page.value_counts(dropna=False))

### Différence des abonnés dans comande et dans sub

In [ ]:
s=set(liste_nouveaux_sub_commande_2015)
t=set(liste_nouveaux_sub_2015)
print "taille de sub commande" , len(s)
print "taille de sub", len(t)
print "commande est il subset de sub ?", s.issubset(t)#	s <= t	test whether every element in s is in t
print "commande est il superset de sub ?", s.issuperset(t)#	s >= t	test whether every element in t is in s
print "taille de l'intersection", len(s.intersection(t))#	s & t	new set with elements common to s and t
print "elements dans commande mais pas dans sub" , len(s.difference(t))# 	s - t	new set with elements in s but not in t
print "elements dans sub mais pas dans commande" , len(t.difference(s))
#s.symmetric_difference(t)

In [ ]:
liste_nouveaux_sub_commande_2015_2 = df_commande.ClientUserId.unique()
liste_nouveaux_sub_2015_2 = df_prod42_test.ClientUserId.unique()
s=set(liste_nouveaux_sub_commande_2015_2)
t=set(liste_nouveaux_sub_2015_2)
print "taille de sub commande : " , len(s)
print "taille de sub : ", len(t)
print "commande est il subset de sub ?", s.issubset(t)#	s <= t	test whether every element in s is in t
print "commande est il superset de sub ?", s.issuperset(t)#	s >= t	test whether every element in t is in s
print "taille de l'intersection : ", len(s.intersection(t))#	s & t	new set with elements common to s and t
print "elements dans commande mais pas dans sub : " , len(s.difference(t))# 	s - t	new set with elements in s but not in t
print "elements dans sub mais pas dans commande : " , len(t.difference(s))
print "\n"

liste_nouveaux_sub_commande_juin_2015_2 = df_comm_juin.ClientUserId.unique()
liste_nouveaux_sub_juin_2015_2 = nouveaux_sub_juin.ClientUserId.unique()

s=set(liste_nouveaux_sub_commande_juin_2015_2)
t=set(liste_nouveaux_sub_juin_2015_2)
print "taille de sub commande : " , len(s)
print "taille de sub : ", len(t)
print "commande est il subset de sub ?", s.issubset(t)#	s <= t	test whether every element in s is in t
print "commande est il superset de sub ?", s.issuperset(t)#	s >= t	test whether every element in t is in s
print "taille de l'intersection : ", len(s.intersection(t))#	s & t	new set with elements common to s and t
print "elements dans commande mais pas dans sub : " , len(s.difference(t))# 	s - t	new set with elements in s but not in t
print "elements dans sub mais pas dans commande : " , len(t.difference(s))

#s.symmetric_difference(t)

### Préparation Kick-off

In [7]:
dfp = pd.read_pickle(path+"\s_dataframe_general_vel_PROD42.pkl")
dfp['SubscriptionCreated'] = pd.to_datetime(dfp['SubscriptionCreated'], format='%d/%m/%Y %H:%M:%S')
dfp['SubscriptionLastUpdated'] = pd.to_datetime(dfp['SubscriptionLastUpdated'], format='%d/%m/%Y %H:%M:%S')
dfp['ServiceExpiry'] = pd.to_datetime(dfp['ServiceExpiry'], format='%d/%m/%Y %H:%M:%S')
print 'ensemble des données prod42 :'
print "min sub creation" ,dfp.SubscriptionCreated.min()
print "max sub creation" ,dfp.SubscriptionCreated.max()
print 'min last update',dfp.SubscriptionLastUpdated.min()
print 'max last update' ,dfp.SubscriptionLastUpdated.max()
print '\n'
print "sites de l'équipe pour prod42 :"
dfp = dfp.loc[dfp['ClientUserId'].isin(liste_client)]
print "min sub creation" ,dfp.SubscriptionCreated.min()
print "max sub creation" ,dfp.SubscriptionCreated.max()
print 'min last update',dfp.SubscriptionLastUpdated.min()
print 'max last update' ,dfp.SubscriptionLastUpdated.max()

ensemble des données prod42
min sub creation 2014-04-23 15:53:59
max sub creation 2015-07-14 23:05:09
min last update 2014-07-21 00:00:19
max last update 2015-07-14 23:47:55


sites de l'équipe pour prod42
min sub creation 2014-04-28 09:52:00
max sub creation 2015-07-14 23:05:09
min last update 2014-07-21 00:00:19
max last update 2015-07-14 23:39:33


In [35]:
dfc = pd.read_pickle(r"W:\L_equipe\s_dataframe_general_xiti_clients.pkl")
dfc['date'] = dfc['fileid'].str.extract(('.+_(.+)\.csv'))

print 'ensemble des données client :'
print "min fileid" ,dfc.date.min()[:4]+"-"+dfc.date.min()[4:6]+"-"+dfc.date.min()[6:]
print "max fileid" ,dfc.date.max()[:4]+"-"+dfc.date.max()[4:6]+"-"+dfc.date.max()[6:]
print '\n'
print "sites de l'équipe pour client :"
dfc = dfc.loc[dfc['id_client'].isin(liste_client)]
print "min fileid" ,dfc.date.min()[:4]+"-"+dfc.date.min()[4:6]+"-"+dfc.date.min()[6:]
print "max fileid" ,dfc.date.max()[:4]+"-"+dfc.date.max()[4:6]+"-"+dfc.date.max()[6:]

ensemble des données client :
min fileid 2013-12-03
max fileid 2015-06-30


sites de l'équipe pour client :
min fileid 2013-12-03
max fileid 2015-06-30


Pour les sites de l'equipe <br/>
date de premiere visite min 2013-12-02 00:00:00<br/>
date de premiere visite max 2015-06-30 00:00:00

In [32]:
#dfco= pd.read_pickle(path+"\s_dataframe_general_vel_commande.pkl")
dfco['OrderDate'] = pd.to_datetime(dfco['OrderDate'], format='%d/%m/%Y %H:%M:%S')
print 'ensemble des données commande : '
print "min orderdate", dfco.OrderDate.min()
print 'max orderdate', dfco.OrderDate.max()
print '\n'
print "sites de l'équipe pour commande : "
dfco = dfco.loc[dfco['ClientUserId'].isin(liste_client)]
print "min fileid" ,dfco.OrderDate.min()
print "max fileid" ,dfco.OrderDate.max()


ensemble des données commande : 
min orderdate 2015-04-07 00:08:52
max orderdate 2015-07-05 23:56:57


sites de l'équipe pour commande : 
min fileid 2015-04-07 00:08:52
max fileid 2015-07-05 23:56:57


ensemble des données pages<br/>
date min : 2013-09-09<br/>
date max : 2015-07-08

ensembe des données sessions<br/>
date min : 2013-09-09<br/>
date max : 2015-06-30<br/>


In [45]:
dfp= pd.read_pickle(path+"\s_dataframe_general_vel_souscription.pkl")
dfp['SubscriptionCreated'] = pd.to_datetime(dfp['SubscriptionCreated'], format='%d/%m/%Y %H:%M:%S')
dfp['SubscriptionLastUpdated'] = pd.to_datetime(dfp['SubscriptionLastUpdated'], format='%d/%m/%Y %H:%M:%S')
dfp['ServiceExpiry'] = pd.to_datetime(dfp['ServiceExpiry'], format='%d/%m/%Y %H:%M:%S')
print 'ensemble des données souscription :'
print "min sub creation" ,dfp.SubscriptionCreated.min()
print "max sub creation" ,dfp.SubscriptionCreated.max()
print 'min last update',dfp.SubscriptionLastUpdated.min()
print 'max last update' ,dfp.SubscriptionLastUpdated.max()
print '\n'
print "sites de l'équipe pour souscription :"
dfp = dfp.loc[dfp['ClientUserId'].isin(liste_client)]
print "min sub creation" ,dfp.SubscriptionCreated.min()
print "max sub creation" ,dfp.SubscriptionCreated.max()
print 'min last update',dfp.SubscriptionLastUpdated.min()
print 'max last update' ,dfp.SubscriptionLastUpdated.max()

ensemble des données souscription :
min sub creation 2014-09-20 21:02:27
max sub creation 2015-07-05 21:53:22
min last update 2015-04-07 00:01:32
max last update 2015-07-05 23:56:58


sites de l'équipe pour souscription :
min sub creation 2014-09-20 21:02:27
max sub creation 2015-07-05 21:46:22
min last update 2015-04-07 00:01:32
max last update 2015-07-05 23:56:58


In [39]:
df_sub_i = pd.read_csv(r"Z:\souscrip_2015.csv", sep=';')
liste_client = pd.read_csv(r"Z:\client_2015.csv", sep=';')
df_sub_clean = pd.read_csv(r"Z:\souscrip_2015_clean.csv", sep=";")
liste_client = liste_client.clientuserid.unique()

In [36]:
dftest = df_sub_i.loc[df_sub_i.clientuserid.isin(liste_client)]

In [38]:
dftest

,fileid,subscriptionid,serviceid,subscriptioncreated,subscriptionlastupdated,subscriptionstatusid,subscriptionstatus,servicetitle,servicedescription,servicegroup,servicegrouptitle,servicegroupdescription,uki,accountid,clientuserid,serviceexpiry,autorenew,explicitpaymenttype,explicitprice,explicitcurrency,pricebandpaymenttype,pricebandamount,pricebandcurrency,istrial,Type_souscription,Montant_essai,Duree_essai,Montant_tacite,Temps,temps_mois,temps_jours,duree_sous,type_promo,mois_souscrip,mois_souscrip_maj,mois_souscrip_exp
0,_EQP_Subscriptions-02012015.csv,3949118,11748,01JAN2015,01JAN2015,2,Active Subscription,Abo_Premium_1�_puis_15.99�,Abonnement Premium : 1€ le 1er mois puis 15.99...,Abos EQP,Abo EQP,Tous les abonnements L'Equipe,jtra@up2news.com,6700672,1428685,01FEB2015,1,eWallet,15.99,EUR,NaN,NaN,NaN,0,directe,1,1 Mois,15.99,1,1.0,30,31,Avec Promo,1,1,2
1,_EQP_Subscriptions-02022015.csv,3949118,11748,01JAN2015,01FEB2015,2,Active Subscription,Abo_Premium_1€_puis_15.99€,Abonnement Premium : 1€ le 1er mois puis 15.99...,Abos EQP,Abo EQP,Tous les abonnements L'Equipe,jtra@up2news.com,6700672,1428685,01MAR2015,1,eWallet,15.99,EUR,NaN,NaN,NaN,0,directe,1,1 Mois,15.99,1,1.0,30,59,Hors promo,1,2,3
2,_EQP_Subscriptions-02032015.csv,3949118,11748,01JAN2015,01MAR2015,2,Active Subscription,Abo_Premium_1€_puis_15.99€,Abonnement Premium : 1€ le 1er mois puis 15.99...,Abos EQP,Abo EQP,Tous les abonnements L'Equipe,jtra@up2news.com,6700672,1428685,01APR2015,1,eWallet,15.99,EUR,NaN,NaN,NaN,0,directe,1,1 Mois,15.99,1,1.0,30,90,Hors promo,1,3,4
3,_EQP_Subscriptions-02042015.csv,3949118,11748,01JAN2015,01APR2015,2,Active Subscription,Abo_Premium_1€_puis_15.99€,Abonnement Premium : 1€ le 1er mois puis 15.99...,Abos EQP,Abo EQP,Tous les abonnements L'Equipe,jtra@up2news.com,6700672,1428685,01MAY2015,1,eWallet,15.99,EUR,NaN,NaN,NaN,0,directe,1,1 Mois,15.99,1,1.0,30,120,Hors promo,1,4,5
4,_EQP_Subscriptions-02052015.csv,3949118,11748,01JAN2015,01MAY2015,2,Active Subscription,Abo_Premium_1€_puis_15.99€,Abonnement Premium : 1€ le 1er mois puis 15.99...,Abos EQP,Abo EQP,Tous les abonnements L'Equipe,jtra@up2news.com,6700672,1428685,01JUN2015,1,eWallet,15.99,EUR,NaN,NaN,NaN,0,directe,1,1 Mois,15.99,1,1.0,30,151,Hors promo,1,5,6
5,_EQP_Subscriptions-02062015.csv,3949118,11748,01JAN2015,01JUN2015,2,Active Subscription,Abo_Premium_1€_puis_15.99€,Abonnement Premium : 1€ le 1er mois puis 15.99...,Abos EQP,Abo EQP,Tous les abonnements L'Equipe,jtra@up2news.com,6700672,1428685,01JUL2015,1,eWallet,15.99,EUR,NaN,NaN,NaN,0,directe,1,1 Mois,15.99,1,1.0,30,181,Hors promo,1,6,7
6,_EQP_Subscriptions-02072015.csv,3949118,11748,01JAN2015,01JUL2015,2,Active Subscription,Abo_Premium_1€_puis_15.99€,Abonnement Premium : 1€ le 1er mois puis 15.99...,Abos EQP,Abo EQP,Tous les abonnements L'Equipe,jtra@up2news.com,6700672,1428685,01AUG2015,1,eWallet,15.99,EUR,NaN,NaN,NaN,0,directe,1,1 Mois,15.99,1,1.0,30,212,Hors promo,1,7,8
7,_EQP_Subscriptions-02082015.csv,3949118,11748,01JAN2015,01AUG2015,2,Active Subscription,Abo_Premium_1€_puis_15.99€,Abonnement Premium : 1€ le 1er mois puis 15.99...,Abos EQP,Abo EQP,Tous les abonnements L'Equipe,jtra@up2news.com,6700672,1428685,01SEP2015,1,eWallet,15.99,EUR,NaN,NaN,NaN,0,directe,1,1 Mois,15.99,1,1.0,30,243,Hors promo,1,8,9
8,_EQP_Subscriptions-02092015.csv,3949118,11748,01JAN2015,01SEP2015,2,Active Subscription,Abo_Premium_1€_puis_15.99€,Abonnement Premium : 1€ le 1er mois puis 15.99...,Abos EQP,Abo EQP,Tous les abonnements L'Equipe,jtra@up2news.com,6700672,1428685,01OCT2015,1,eWallet,15.99,EUR,NaN,NaN,NaN,0,directe,1,1 Mois,15.99,1,1.0,30,273,Hors promo,1,9,10
9,_EQP_Subscriptions-02102015.csv,3949118,11748,01JAN2015,01OCT2015,2,Active Subscription,Abo_Premium_1€_puis_15.99€,Abonnement Premium : 1€ le 1er mois puis 15.99...,Abos EQP,Abo EQP,Tous les abonnements L'Equipe,jtra@up2news.com,6700672,1428685,01NOV2015,1,eWallet,15.99,EUR,NaN,NaN,NaN,0,directe,1,1 Mois,15.99,1,1.0,30,304,Hors promo,1,10,11


In [17]:
dftest = dftest.drop_duplicates(['subscriptionid'])

In [44]:
dftest.loc[((dftest.subscriptionstatusid ==1) & (dftest.type_promo == 'Avec Promo'))]

,fileid,subscriptionid,serviceid,subscriptioncreated,subscriptionlastupdated,subscriptionstatusid,subscriptionstatus,servicetitle,servicedescription,servicegroup,servicegrouptitle,servicegroupdescription,uki,accountid,clientuserid,serviceexpiry,autorenew,explicitpaymenttype,explicitprice,explicitcurrency,pricebandpaymenttype,pricebandamount,pricebandcurrency,istrial,Type_souscription,Montant_essai,Duree_essai,Montant_tacite,Temps,temps_mois,temps_jours,duree_sous,type_promo,mois_souscrip,mois_souscrip_maj,mois_souscrip_exp
1710,_EQP_Subscriptions-06032015.csv,3987995,11888,02FEB2015,05MAR2015,1,Cancelled By Customer Support Agent,Abo_BeIN_L'Equipe_Promo_2mois_1€_puis_15.99€,"Pour tout abonnement à BeIN, abonnement de 2 m...",Abos EQP,Abo EQP,Tous les abonnements L'Equipe,nab.youssef@yahoo.fr,6785581,1453572,05MAR2015,0,eWallet,15.99,EUR,NaN,NaN,NaN,0,directe,1,2 Mois,15.99,2,2.0,60,31,Avec Promo,2,3,3
3357,_EQP_Subscriptions-12032015.csv,4062196,11762,27FEB2015,11MAR2015,1,Cancelled By Customer Support Agent,"Abo Premium Promo 1€ puis 11,99€",Abonnement Premium : Promo 1€ le 1er mois puis...,Abos EQP,Abo EQP,Tous les abonnements L'Equipe,barbaguy@numericable.fr,5946146,808415,11MAR2015,0,eWallet,11.99,EUR,NaN,NaN,NaN,0,directe,1,1 Mois,11.99,1,1.0,30,12,Avec Promo,2,3,3
3570,_EQP_Subscriptions-04032015.csv,4065278,11748,02MAR2015,03MAR2015,1,Cancelled By Customer Support Agent,Abo_Premium_1€_puis_15.99€,Abonnement Premium : 1€ le 1er mois puis 15.99...,Abos EQP,Abo EQP,Tous les abonnements L'Equipe,nilcoas@yahoo.fr,6922460,1473128,03MAR2015,0,eWallet,15.99,EUR,NaN,NaN,NaN,0,directe,1,1 Mois,15.99,1,1.0,30,1,Avec Promo,3,3,3
3857,_EQP_Subscriptions-11032015.csv,4072286,11748,08MAR2015,10MAR2015,1,Cancelled By Customer Support Agent,Abo_Premium_1€_puis_15.99€,Abonnement Premium : 1€ le 1er mois puis 15.99...,Abos EQP,Abo EQP,Tous les abonnements L'Equipe,barbaguy@numericable.fr,5946146,808415,10MAR2015,0,eWallet,15.99,EUR,NaN,NaN,NaN,0,directe,1,1 Mois,15.99,1,1.0,30,2,Avec Promo,3,3,3
5588,_EQP_Subscriptions-09042015.csv,4113169,11748,08APR2015,08APR2015,1,Cancelled By Customer Support Agent,Abo_Premium_1€_puis_15.99€,Abonnement Premium : 1€ le 1er mois puis 15.99...,Abos EQP,Abo EQP,Tous les abonnements L'Equipe,20150408.pf@yruama.net,7069834,1500600,08APR2015,0,eWallet,15.99,EUR,NaN,NaN,NaN,0,directe,1,1 Mois,15.99,1,1.0,30,0,Avec Promo,4,4,4
11276,_EQP_Subscriptions-11062015.csv,4166540,12546,23MAY2015,10JUN2015,1,Cancelled By Customer Support Agent,Sprint_final_L1_1mois_offert_puis_9.99€,Du 4 au 23 mai - Sprint final L1 : 1 mois offe...,Abos EQP,Abo EQP,Tous les abonnements L'Equipe,arnaud.mezrahi@gmail.com,6696523,1427142,10JUN2015,0,eWallet,9.99,EUR,NaN,NaN,NaN,0,directe,0,1 Mois,9.99,1,1.0,30,18,Avec Promo,5,6,6
11353,_EQP_Subscriptions-26052015.csv,4166783,12546,23MAY2015,25MAY2015,1,Cancelled By Customer Support Agent,Sprint_final_L1_1mois_offert_puis_9.99€,Du 4 au 23 mai - Sprint final L1 : 1 mois offe...,Abos EQP,Abo EQP,Tous les abonnements L'Equipe,lucpetin@gmail.com,5948735,532695,25MAY2015,0,eWallet,9.99,EUR,NaN,NaN,NaN,0,directe,0,1 Mois,9.99,1,1.0,30,2,Avec Promo,5,5,5
11884,_EQP_Subscriptions-30052015.csv,4170094,12567,27MAY2015,29MAY2015,1,Cancelled By Customer Support Agent,RG2015_1€_15jrs_puis_11.99€,Du 24 mai au 7 juin 2015 - Roland Garros : 15 ...,Abos EQP,Abo EQP,Tous les abonnements L'Equipe,you.lazrak@gmail.com,7217778,599287,29MAY2015,0,eWallet,11.99,EUR,NaN,NaN,NaN,0,directe,1,15 Jours,11.99,15,0.5,15,2,Avec Promo,5,5,5
15742,_EQP_Subscriptions-07072015.csv,4283306,11748,26JUN2015,06JUL2015,1,Cancelled By Customer Support Agent,Abo_Premium_1€_puis_15.99€,Abonnement Premium : 1€ le 1er mois puis 15.99...,Abos EQP,Abo EQP,Tous les abonnements L'Equipe,stephaneradiguet08@gmail.com,5944474,947415,06JUL2015,0,eWallet,15.99,EUR,NaN,NaN,NaN,0,directe,1,1 Mois,15.99,1,1.0,30,10,Avec Promo,6,7,7
17297,_EQP_Subscriptions-07082015.csv,4297113,12596,05JUL2015,06AUG2015,1,Cancelled By Customer Support Agent,Eté_2

In [40]:
df_sub_clean

,fileid,subscriptionid,serviceid,subscriptioncreated,subscriptionlastupdated,subscriptionstatusid,subscriptionstatus,servicetitle,servicedescription,servicegroup,servicegrouptitle,servicegroupdescription,uki,accountid,clientuserid,serviceexpiry,autorenew,explicitpaymenttype,explicitprice,explicitcurrency,pricebandpaymenttype,pricebandamount,pricebandcurrency,istrial,Type_souscription,Montant_essai,Duree_essai,Montant_tacite,Temps,temps_mois,temps_jours,duree_sous,type_promo,mois_souscrip,mois_souscrip_maj,mois_souscrip_exp
0,_EQP_Subscriptions-02012015.csv,3949118,11748,01JAN2015,01JAN2015,2,Active Subscription,Abo_Premium_1�_puis_15.99�,Abonnement Premium : 1€ le 1er mois puis 15.99...,Abos EQP,Abo EQP,Tous les abonnements L'Equipe,jtra@up2news.com,6700672,1428685,01FEB2015,1,eWallet,15.99,EUR,NaN,NaN,NaN,0,directe,1,1 Mois,15.99,1,1.0,30,31,Avec Promo,1,1,2
1,_EQP_Subscriptions-02022015.csv,3949118,11748,01JAN2015,01FEB2015,2,Active Subscription,Abo_Premium_1€_puis_15.99€,Abonnement Premium : 1€ le 1er mois puis 15.99...,Abos EQP,Abo EQP,Tous les abonnements L'Equipe,jtra@up2news.com,6700672,1428685,01MAR2015,1,eWallet,15.99,EUR,NaN,NaN,NaN,0,directe,1,1 Mois,15.99,1,1.0,30,59,Hors promo,1,2,3
2,_EQP_Subscriptions-02032015.csv,3949118,11748,01JAN2015,01MAR2015,2,Active Subscription,Abo_Premium_1€_puis_15.99€,Abonnement Premium : 1€ le 1er mois puis 15.99...,Abos EQP,Abo EQP,Tous les abonnements L'Equipe,jtra@up2news.com,6700672,1428685,01APR2015,1,eWallet,15.99,EUR,NaN,NaN,NaN,0,directe,1,1 Mois,15.99,1,1.0,30,90,Hors promo,1,3,4
3,_EQP_Subscriptions-02042015.csv,3949118,11748,01JAN2015,01APR2015,2,Active Subscription,Abo_Premium_1€_puis_15.99€,Abonnement Premium : 1€ le 1er mois puis 15.99...,Abos EQP,Abo EQP,Tous les abonnements L'Equipe,jtra@up2news.com,6700672,1428685,01MAY2015,1,eWallet,15.99,EUR,NaN,NaN,NaN,0,directe,1,1 Mois,15.99,1,1.0,30,120,Hors promo,1,4,5
4,_EQP_Subscriptions-02052015.csv,3949118,11748,01JAN2015,01MAY2015,2,Active Subscription,Abo_Premium_1€_puis_15.99€,Abonnement Premium : 1€ le 1er mois puis 15.99...,Abos EQP,Abo EQP,Tous les abonnements L'Equipe,jtra@up2news.com,6700672,1428685,01JUN2015,1,eWallet,15.99,EUR,NaN,NaN,NaN,0,directe,1,1 Mois,15.99,1,1.0,30,151,Hors promo,1,5,6
5,_EQP_Subscriptions-02062015.csv,3949118,11748,01JAN2015,01JUN2015,2,Active Subscription,Abo_Premium_1€_puis_15.99€,Abonnement Premium : 1€ le 1er mois puis 15.99...,Abos EQP,Abo EQP,Tous les abonnements L'Equipe,jtra@up2news.com,6700672,1428685,01JUL2015,1,eWallet,15.99,EUR,NaN,NaN,NaN,0,directe,1,1 Mois,15.99,1,1.0,30,181,Hors promo,1,6,7
6,_EQP_Subscriptions-02072015.csv,3949118,11748,01JAN2015,01JUL2015,2,Active Subscription,Abo_Premium_1€_puis_15.99€,Abonnement Premium : 1€ le 1er mois puis 15.99...,Abos EQP,Abo EQP,Tous les abonnements L'Equipe,jtra@up2news.com,6700672,1428685,01AUG2015,1,eWallet,15.99,EUR,NaN,NaN,NaN,0,directe,1,1 Mois,15.99,1,1.0,30,212,Hors promo,1,7,8
7,_EQP_Subscriptions-02082015.csv,3949118,11748,01JAN2015,01AUG2015,2,Active Subscription,Abo_Premium_1€_puis_15.99€,Abonnement Premium : 1€ le 1er mois puis 15.99...,Abos EQP,Abo EQP,Tous les abonnements L'Equipe,jtra@up2news.com,6700672,1428685,01SEP2015,1,eWallet,15.99,EUR,NaN,NaN,NaN,0,directe,1,1 Mois,15.99,1,1.0,30,243,Hors promo,1,8,9
8,_EQP_Subscriptions-02092015.csv,3949118,11748,01JAN2015,01SEP2015,2,Active Subscription,Abo_Premium_1€_puis_15.99€,Abonnement Premium : 1€ le 1er mois puis 15.99...,Abos EQP,Abo EQP,Tous les abonnements L'Equipe,jtra@up2news.com,6700672,1428685,01OCT2015,1,eWallet,15.99,EUR,NaN,NaN,NaN,0,directe,1,1 Mois,15.99,1,1.0,30,273,Hors promo,1,9,10
9,_EQP_Subscriptions-02102015.csv,3949118,11748,01JAN2015,01OCT2015,2,Active Subscription,Abo_Premium_1€_puis_15.99€,Abonnement Premium : 1€ le 1er mois puis 15.99...,Abos EQP,Abo EQP,Tous les abonnements L'Equipe,jtra@up2news.com,6700672,1428685,01NOV2015,1,eWallet,15.99,EUR,NaN,NaN,NaN,0,directe,1,1 Mois,15.99,1,1.0,30,304,Hors promo,1,10,11


In [50]:
liste_sous = df_sub_clean.loc[((df_sub_clean.subscriptionstatusid ==1) & (df_sub_clean.type_promo == 'Avec Promo'))].subscriptionid.unique()
print liste_sous

[4319832 4323641 4400831 4403559]


In [49]:
df_sub_clean.loc[df_sub_clean.subscriptionid.isin(liste_sous)].sort_values(by="subscriptionid")

,fileid,subscriptionid,serviceid,subscriptioncreated,subscriptionlastupdated,subscriptionstatusid,subscriptionstatus,servicetitle,servicedescription,servicegroup,servicegrouptitle,servicegroupdescription,uki,accountid,clientuserid,serviceexpiry,autorenew,explicitpaymenttype,explicitprice,explicitcurrency,pricebandpaymenttype,pricebandamount,pricebandcurrency,istrial,Type_souscription,Montant_essai,Duree_essai,Montant_tacite,Temps,temps_mois,temps_jours,duree_sous,type_promo,mois_souscrip,mois_souscrip_maj,mois_souscrip_exp
18495,_EQP_Subscriptions-27072015.csv,4319832,12596,26JUL2015,26JUL2015,2,Active Subscription,Eté_2015_0€_puis_11.99€,OP Eté 2015 : 0€ le 1er mois puis 11.99€/mois ...,Abos EQP,Abo EQP,Tous les abonnements L'Equipe,bordenvave.maurice@aliceadsl.fr,7504118,1573428,26AUG2015,1,eWallet,11.99,EUR,NaN,NaN,NaN,0,directe,0,1 Mois,11.99,1,1.0,30,31,Avec Promo,7,7,8
18496,_EQP_Subscriptions-28072015.csv,4319832,12596,26JUL2015,27JUL2015,1,Cancelled By Customer Support Agent,Eté_2015_0€_puis_11.99€,OP Eté 2015 : 0€ le 1er mois puis 11.99€/mois ...,Abos EQP,Abo EQP,Tous les abonnements L'Equipe,bordenvave.maurice@aliceadsl.fr,7504118,1573428,27JUL2015,0,eWallet,11.99,EUR,NaN,NaN,NaN,0,directe,0,1 Mois,11.99,1,1.0,30,1,Avec Promo,7,7,7
18497,_EQP_Subscriptions-29072015.csv,4319832,12596,26JUL2015,28JUL2015,2,Active Subscription,Eté_2015_0€_puis_11.99€,OP Eté 2015 : 0€ le 1er mois puis 11.99€/mois ...,Abos EQP,Abo EQP,Tous les abonnements L'Equipe,bordenvave.maurice@aliceadsl.fr,7504118,1573428,28AUG2015,1,eWallet,11.99,EUR,NaN,NaN,NaN,0,directe,0,1 Mois,11.99,1,1.0,30,33,Hors promo,7,7,8
18498,_EQP_Subscriptions-29082015.csv,4319832,12596,26JUL2015,28AUG2015,2,Active Subscription,Eté_2015_0€_puis_11.99€,OP Eté 2015 : 0€ le 1er mois puis 11.99€/mois ...,Abos EQP,Abo EQP,Tous les abonnements L'Equipe,bordenvave.maurice@aliceadsl.fr,7504118,1573428,28SEP2015,1,eWallet,11.99,EUR,NaN,NaN,NaN,0,directe,0,1 Mois,11.99,1,1.0,30,64,Hors promo,7,8,9
18499,_EQP_Subscriptions-29092015.csv,4319832,12596,26JUL2015,28SEP2015,2,Active Subscription,Eté_2015_0€_puis_11.99€,OP Eté 2015 : 0€ le 1er mois puis 11.99€/mois ...,Abos EQP,Abo EQP,Tous les abonnements L'Equipe,bordenave.maurice@aliceadsl.fr,7504118,1573428,28OCT2015,1,eWallet,11.99,EUR,NaN,NaN,NaN,0,directe,0,1 Mois,11.99,1,1.0,30,94,Hors promo,7,9,10
18500,_EQP_Subscriptions-29102015.csv,4319832,12596,26JUL2015,28OCT2015,2,Active Subscription,Eté_2015_0€_puis_11.99€,OP Eté 2015 : 0€ le 1er mois puis 11.99€/mois ...,Abos EQP,Abo EQP,Tous les abonnements L'Equipe,bordenave.maurice@aliceadsl.fr,7504118,1573428,28NOV2015,1,eWallet,11.99,EUR,NaN,NaN,NaN,0,directe,0,1 Mois,11.99,1,1.0,30,125,Hors promo,7,10,11
18501,_EQP_Subscriptions-29112015.csv,4319832,12596,26JUL2015,28NOV2015,2,Active Subscription,Eté_2015_0€_puis_11.99€,OP Eté 2015 : 0€ le 1er mois puis 11.99€/mois ...,Abos EQP,Abo EQP,Tous les abonnements L'Equipe,bordenave.maurice@aliceadsl.fr,7504118,1573428,28DEC2015,1,eWallet,11.99,EUR,NaN,NaN,NaN,0,directe,0,1 Mois,11.99,1,1.0,30,155,Hors promo,7,11,12
18502,_EQP_Subscriptions-29122015.csv,4319832,12596,26JUL2015,28DEC2015,2,Active Subscription,Eté_2015_0€_puis_11.99€,OP Eté 2015 : 0€ le 1er mois puis 11.99€/mois ...,Abos EQP,Abo EQP,Tous les abonnements L'Equipe,bordenave.maurice@aliceadsl.fr,7504118,1573428,31DEC2015,1,eWallet,11.99,EUR,NaN,NaN,NaN,0,directe,0,1 Mois,11.99,1,1.0,30,158,Hors promo,7,12,12
18966,_EQP_Subscriptions-17082015.csv,4323641,12596,29JUL2015,16AUG2015,1,Cancelled By Customer Support Agent,Eté_2015_0€_puis_11.99€,OP Eté 2015 : 0€ le 1er mois puis 11.99€/mois ...,Abos EQP,Abo EQP,Tous les abonnements L'Equipe,cyrille.lokocki@sfr.fr,7514953,1575577,16AUG2015,0,eWallet,11.99,EUR,NaN,NaN,NaN,0,directe,0,1 Mois,11.99,1,1.0,30,18,Avec Promo,7,8,8
18967,_EQP_Subscriptions-18082015.csv,4323641,12596,29JUL2015,17AUG2015,2,Active Subscription,Eté_2015_0€_puis_11.99€,OP Eté 2015 : 0€ le 1er mois puis 11.99€/mois ...,Abos EQP,Abo EQP,Tous les abonnements L'Equi